In [1]:
import os
import pandas as pd 
import numpy as np
from keras.callbacks import *

Using TensorFlow backend.


In [2]:
path = 'train/'
paths = ['验证码/train/','验证码/train_dbd/','验证码/train_ld/','验证码/train_rd/','验证码/train_ys/']
datas = pd.read_csv(path+'train_label.csv')

In [3]:
datas.head(n=5)

,ID,label
0,1.jpg,4Jy3
1,2.jpg,P6Bf
2,3.jpg,vJlD
3,4.jpg,1Mnb
4,5.jpg,B6UB


In [4]:
image_list = list(datas['ID'])
image_label = list(datas['label'])

# 训练及数据加载

In [5]:
import numpy as np
import os

from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input, concatenate ,BatchNormalization
from keras.layers.convolutional import Conv2D, Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta
from keras.utils.vis_utils import plot_model
import tensorflow as tf
import glob

# 验证码所包含的字符 _表示未知
captcha_word = ['0','1','2','3','4','5','6','7','8','9',
           'A','B','C','D','E','F','G','H','I','J',
           'K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
           'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p',
           'q','r','s','t','u','v','w','x','y','z'
           ]

# 图片的长度和宽度
width = 120
height = 40

# 每个验证码所包含的字符数
word_len = 4
# 字符总数
word_class = len(captcha_word)

samples = glob.glob(r'train/*.jpg')

In [6]:
# 生成字符索引，同时反向操作一次，方面还原
char_indices = dict((c, i) for i, c in enumerate(captcha_word))
indices_char = dict((i, c) for i, c in enumerate(captcha_word))

In [7]:
# 验证码字符串转数组
def captcha_to_vec(captcha):
    # 创建一个长度为 字符个数 * 字符种数 长度的数组
    vector = np.zeros(word_len * word_class)

    # 文字转成成数组
    for i, ch in enumerate(captcha):
        idex = i * word_class + char_indices[ch]
        vector[idex] = 1
    return vector

In [8]:
# 把数组转换回文字
def vec_to_captcha(vec):
    text = []
    # 把概率小于0.5的改为0，标记为错误
    vec[vec < 0.5] = 0

    char_pos = vec.nonzero()[0]

    for i, ch in enumerate(char_pos):
        text.append(captcha_word[ch % word_class])
    return ''.join(text)

In [9]:
# 自定义评估函数
def custom_accuracy(y_true, y_pred):
    predict = tf.reshape(y_pred, [-1, word_len, word_class])
    max_idx_p = tf.argmax(predict, 2)#这个做法牛逼，不用再做stack和reshape了，2，是在Charset那个维度上
    max_idx_l = tf.argmax(tf.reshape(y_true, [-1, word_len,word_class]), 2)
    correct_pred = tf.equal(max_idx_p, max_idx_l)
    _result = tf.map_fn(fn=lambda e: tf.reduce_all(e),elems=correct_pred,dtype=tf.bool)
    return tf.reduce_mean(tf.cast(_result, tf.float32))

#获取目录下样本列表
#创建数组，储存图片信息。样本个数、宽度和高度。
# 3代表图片的通道数，如果对图片进行了灰度处理，可以改为单通道 1


In [10]:
X = np.zeros((len(image_list), height, width, 3), dtype = np.uint8)
# 创建数组，储存标签信息
y = np.zeros((len(image_label), word_len * word_class), dtype = np.uint8)
for i,img in enumerate(image_list):  
    if i % 10000 == 0:
        print(i,img)
        print(image_label[i])
    img_path = path + "/" + img
    #读取图片
    raw_img = image.load_img(img_path, target_size=(height, width))
    #讲图片转为np数组
    X[i] = image.img_to_array(raw_img)
    #讲标签转换为数组进行保存
    y[i] = captcha_to_vec(image_label[i])

0 1.jpg
4Jy3


In [11]:
X = np.zeros((len(image_list)*5, height, width, 3), dtype = np.uint8)
# 创建数组，储存标签信息
y = np.zeros((len(image_label)*5, word_len * word_class), dtype = np.uint8)
image_labels = []
p = 0
for j in paths:
    print(j) 
    for i,img in enumerate(image_list):  
        if p % 2500 == 0:
            print(i,img)
            print(image_label[i])
        p += 1
        img_path = j + "/" + img
        #读取图片
        raw_img = image.load_img(img_path, target_size=(height, width))
        #讲图片转为np数组
        X[i] = image.img_to_array(raw_img)
        #讲标签转换为数组进行保存
        image_labels.append(image_label[i])
        y[i] = captcha_to_vec(image_label[i])

验证码/train/
0 1.jpg
4Jy3
2500 2501.jpg
oTZK
验证码/train_dbd/
0 1.jpg
4Jy3
2500 2501.jpg
oTZK
验证码/train_ld/
0 1.jpg
4Jy3
2500 2501.jpg
oTZK
验证码/train_rd/
0 1.jpg
4Jy3
2500 2501.jpg
oTZK
验证码/train_ys/
0 1.jpg
4Jy3
2500 2501.jpg
oTZK


In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(X,y,test_size=0.1,random_state=2019)

# 模型零

In [17]:
from keras.callbacks import *
from keras.optimizers import Adam 
#创建输入，结构为 高，宽，通道
input_tensor = Input( shape=(height, width, 3))

x = input_tensor

#构建卷积网络
#两层卷积层，一层池化层，重复3次。因为生成的验证码比较小，padding使用same
x = Convolution2D(32, 3, padding='same', activation='relu')(x)
x = Convolution2D(64, 3, padding='same', activation='relu')(x)
# x= BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)


x = Convolution2D(32, 3, padding='same', activation='relu')(x)
x = Convolution2D(64, 3, padding='same', activation='relu')(x)
# x= BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Convolution2D(128, 3, padding='same', activation='relu')(x)
x = Convolution2D(128, 3, padding='same',activation='relu')(x)
# x= BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

#Flatten层用来将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡。
x = Flatten()(x)
#为输入数据施加Dropout。Dropout将在训练过程中每次更新参数时随机断开一定百分比（rate）的输入神经元，Dropout层用于防止过拟合。
x = Dropout(0.25)(x)
x= BatchNormalization()(x)
#Dense就是常用的全连接层
#最后连接4个分类器，每个分类器是56个神经元，分别输出56个字符的概率。
x = [Dense(word_class, activation='sigmoid', name='c%d'%(i+1))(x) for i in range(word_len)]
# x = [Dense(word_class, activation='sigmoid', name='c%d'%(i+1))(x) for i in range(word_len)]
output = concatenate(x)
#构建模型
model = Model(inputs=input_tensor, outputs=output)
# model.add(BatchNormalization())
#因为训练可能需要数个小时，所以这里加载了之前我训练好的参数。准确率为94%
#可以直接使用此参数继续进行训练，也可以自己从头开始训练
# model.load_weights('model/weights.10--6.55-0.3062.hdf5')

#这里优化器选用Adadelta，学习率0.1
#编译模型以供训练，损失函数使用 categorical_crossentropy，使用accuracy评估模型在训练和测试时的性能的指标
#这里使用自定义的评估模型
adam = Adam(lr=0.00001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy',custom_accuracy])
#model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics=['accuracy',custom_accuracy])
#每次epoch都保存一下权重，用于继续训练
#early_stopping = EarlyStopping(monitor='val_loss', patience=100)
#plateau = ReduceLROnPlateau(monitor="val_acc", verbose=1, mode='max', factor=0.5, patience=2)
plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, verbose=1, mode='auto')

checkpointer = ModelCheckpoint(filepath="model/model3/cnn_weights.{epoch:02d}--{val_loss:.2f}-{val_acc:.4f}.hdf5", monitor='val_loss', 
#                                 verbose=2, save_best_only=True, mode='min')
#开始训练，validation_split代表10%的数据不参与训练，用于做验证急
#我之前训练了50个epochs以上，这里根据自己的情况进行选择。如果输出的val_acc已经达到你满意的数值，可以终止训练
model.fit(X_train, Y_train, epochs= 1000,callbacks=[plateau], validation_split=0.1)
# plot model

plot_model(model, to_file='model/model3/model.png', show_shapes=True)

#保存权重和模型
model.save_weights('model/model3/cnn_captcha_model_weights.h5')
model.save('model/model3/cnn_captcha__model_2.h5')


Train on 4050 samples, validate on 450 samples
Epoch 1/1000
4050/4050 [==============================] - 6s 1ms/step - loss: 22.8816 - acc: 0.0032 - custom_accuracy: 0.0000e+00 - val_loss: 25.3744 - val_acc: 0.0000e+00 - val_custom_accuracy: 0.0000e+00
Epoch 2/1000
4050/4050 [==============================] - 5s 1ms/step - loss: 22.0236 - acc: 0.0030 - custom_accuracy: 0.0000e+00 - val_loss: 28.1834 - val_acc: 0.0022 - val_custom_accuracy: 0.0000e+00
Epoch 3/1000
4050/4050 [==============================] - 5s 1ms/step - loss: 21.0683 - acc: 0.0232 - custom_accuracy: 0.0000e+00 - val_loss: 23.3924 - val_acc: 0.0044 - val_custom_accuracy: 0.0000e+00
Epoch 4/1000
4050/4050 [==============================] - 5s 1ms/step - loss: 16.8218 - acc: 0.1010 - custom_accuracy: 0.0109 - val_loss: 34.9367 - val_acc: 0.0133 - val_custom_accuracy: 0.0000e+00
Epoch 5/1000
4050/4050 [==============================] - 5s 1ms/step - loss: 13.4112 - acc: 0.1869 - custom_accuracy: 0.0657 - val_loss: 13.9562

Epoch 82/1000
4050/4050 [==============================] - 5s 1ms/step - loss: 5.7714 - acc: 0.2637 - custom_accuracy: 0.9948 - val_loss: 7.3542 - val_acc: 0.2867 - val_custom_accuracy: 0.8111
Epoch 83/1000
4050/4050 [==============================] - 5s 1ms/step - loss: 5.7751 - acc: 0.2694 - custom_accuracy: 0.9931 - val_loss: 7.3498 - val_acc: 0.2933 - val_custom_accuracy: 0.8133
Epoch 84/1000
4050/4050 [==============================] - 5s 1ms/step - loss: 5.7770 - acc: 0.2659 - custom_accuracy: 0.9931 - val_loss: 7.3454 - val_acc: 0.2956 - val_custom_accuracy: 0.8200
Epoch 85/1000
4050/4050 [==============================] - 5s 1ms/step - loss: 5.7688 - acc: 0.2706 - custom_accuracy: 0.9956 - val_loss: 7.4151 - val_acc: 0.2867 - val_custom_accuracy: 0.8178
Epoch 86/1000
4050/4050 [==============================] - 5s 1ms/step - loss: 5.7725 - acc: 0.2686 - custom_accuracy: 0.9928 - val_loss: 7.3333 - val_acc: 0.2911 - val_custom_accuracy: 0.8200
Epoch 87/1000
4050/4050 [==========

KeyboardInterrupt: 

In [19]:
adam = Adam(lr=0.00001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy',custom_accuracy])
#model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics=['accuracy',custom_accuracy])
#每次epoch都保存一下权重，用于继续训练
#early_stopping = EarlyStopping(monitor='val_loss', patience=100)
#plateau = ReduceLROnPlateau(monitor="val_acc", verbose=1, mode='max', factor=0.5, patience=2)
plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, verbose=1, mode='auto')

checkpointer = ModelCheckpoint(filepath="model/model3/cnn_weights.{epoch:02d}--{val_loss:.2f}-{val_acc:.4f}.hdf5", monitor='val_loss', 
                                 verbose=2, save_best_only=True, mode='min')
#开始训练，validation_split代表10%的数据不参与训练，用于做验证急
#我之前训练了50个epochs以上，这里根据自己的情况进行选择。如果输出的val_acc已经达到你满意的数值，可以终止训练
model.fit(X_train, Y_train, epochs= 1000,callbacks=[plateau], validation_split=0.1)

Train on 4050 samples, validate on 450 samples
Epoch 1/1000
4050/4050 [==============================] - 6s 1ms/step - loss: 5.7581 - acc: 0.2632 - custom_accuracy: 0.9933 - val_loss: 7.4423 - val_acc: 0.2800 - val_custom_accuracy: 0.8156
Epoch 2/1000
4050/4050 [==============================] - 5s 1ms/step - loss: 5.7588 - acc: 0.2716 - custom_accuracy: 0.9938 - val_loss: 7.3142 - val_acc: 0.2844 - val_custom_accuracy: 0.8178
Epoch 3/1000
4050/4050 [==============================] - 5s 1ms/step - loss: 5.7530 - acc: 0.2662 - custom_accuracy: 0.9941 - val_loss: 7.6216 - val_acc: 0.2822 - val_custom_accuracy: 0.8133
Epoch 4/1000
4050/4050 [==============================] - 5s 1ms/step - loss: 5.7584 - acc: 0.2677 - custom_accuracy: 0.9941 - val_loss: 7.5736 - val_acc: 0.2822 - val_custom_accuracy: 0.8156
Epoch 5/1000
4050/4050 [==============================] - 5s 1ms/step - loss: 5.7514 - acc: 0.2736 - custom_accuracy: 0.9936 - val_loss: 7.5864 - val_acc: 0.2911 - val_custom_accuracy: 

KeyboardInterrupt: 

# 模型一

In [21]:
#创建输入，结构为 高，宽，通道
input_tensor = Input( shape=(height, width, 3))

x = input_tensor

#构建卷积网络
#两层卷积层，一层池化层，重复3次。因为生成的验证码比较小，padding使用same
x = Convolution2D(32, 3, padding='same', activation='relu')(x)
x = Convolution2D(32, 3, padding='same', activation='relu')(x)
# x= BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)


x = Convolution2D(64, 3, padding='same', activation='relu')(x)
x = Convolution2D(64, 3, padding='same', activation='relu')(x)
# x= BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Convolution2D(64, 3, padding='same', activation='relu')(x)
x = Convolution2D(64, 3, padding='same',activation='relu')(x)
# x= BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Convolution2D(128, 3, padding='same', activation='relu')(x)
x = Convolution2D(128, 3, padding='same',activation='relu')(x)
# x= BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Convolution2D(128, 3, padding='same', activation='relu')(x)
x = Convolution2D(128, 3, padding='same',activation='relu')(x)
# x= BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)


#Flatten层用来将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡。
x = Flatten()(x)
#为输入数据施加Dropout。Dropout将在训练过程中每次更新参数时随机断开一定百分比（rate）的输入神经元，Dropout层用于防止过拟合。
x = Dropout(0.25)(x)
x= BatchNormalization()(x)
#Dense就是常用的全连接层
#最后连接4个分类器，每个分类器是56个神经元，分别输出56个字符的概率。
x = [Dense(word_class, activation='sigmoid', name='c%d'%(i+1))(x) for i in range(word_len)]
# x = [Dense(word_class, activation='sigmoid', name='c%d'%(i+1))(x) for i in range(word_len)]
output = concatenate(x)
#构建模型
model = Model(inputs=input_tensor, outputs=output)
# model.add(BatchNormalization())
#因为训练可能需要数个小时，所以这里加载了之前我训练好的参数。准确率为94%
#可以直接使用此参数继续进行训练，也可以自己从头开始训练
# model.load_weights('model/weights.10--6.55-0.3062.hdf5')

#这里优化器选用Adadelta，学习率0.1
opt = Adadelta(lr=0.1)
#编译模型以供训练，损失函数使用 categorical_crossentropy，使用accuracy评估模型在训练和测试时的性能的指标
#这里使用自定义的评估模型
model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics=['accuracy',custom_accuracy])
#每次epoch都保存一下权重，用于继续训练
early_stopping = EarlyStopping(monitor='val_loss', patience=500)
#plateau = ReduceLROnPlateau(monitor="val_acc", verbose=1, mode='max', factor=0.5, patience=2)
plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.00001, cooldown=0, min_lr=0)

checkpointer = ModelCheckpoint(filepath="model/model2/weights.{epoch:02d}--{val_loss:.2f}-{val_acc:.4f}.hdf5", monitor='val_loss', 
                                 verbose=2, save_best_only=True, mode='min')
#开始训练，validation_split代表10%的数据不参与训练，用于做验证急
#我之前训练了50个epochs以上，这里根据自己的情况进行选择。如果输出的val_acc已经达到你满意的数值，可以终止训练
model.fit(X, y, epochs= 10000,callbacks=[checkpointer], validation_split=0.1)
# plot model
plot_model(model, to_file='model/model2/model.png', show_shapes=True)



Train on 4500 samples, validate on 500 samples
Epoch 1/10000
4500/4500 [==============================] - 6s 1ms/step - loss: 24.1142 - acc: 0.0031 - custom_accuracy: 0.0000e+00 - val_loss: 24.4603 - val_acc: 0.0020 - val_custom_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 24.46030, saving model to model/model2/weights.01--24.46-0.0020.hdf5
Epoch 2/10000
4500/4500 [==============================] - 4s 998us/step - loss: 23.4983 - acc: 0.0038 - custom_accuracy: 0.0000e+00 - val_loss: 23.3316 - val_acc: 0.0060 - val_custom_accuracy: 0.0000e+00

Epoch 00002: val_loss improved from 24.46030 to 23.33156, saving model to model/model2/weights.02--23.33-0.0060.hdf5
Epoch 3/10000
4500/4500 [==============================] - 4s 995us/step - loss: 22.9105 - acc: 0.0042 - custom_accuracy: 0.0000e+00 - val_loss: 23.0653 - val_acc: 0.0020 - val_custom_accuracy: 0.0000e+00

Epoch 00003: val_loss improved from 23.33156 to 23.06525, saving model to model/model2/weights.03--23.07-0.0


Epoch 00029: val_loss did not improve from 22.05521
Epoch 30/10000
4500/4500 [==============================] - 4s 988us/step - loss: 22.0322 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+00 - val_loss: 22.0618 - val_acc: 0.0000e+00 - val_custom_accuracy: 0.0000e+00

Epoch 00030: val_loss did not improve from 22.05521
Epoch 31/10000
4500/4500 [==============================] - 4s 976us/step - loss: 22.0321 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+00 - val_loss: 22.0622 - val_acc: 0.0000e+00 - val_custom_accuracy: 0.0000e+00

Epoch 00031: val_loss did not improve from 22.05521
Epoch 32/10000
4500/4500 [==============================] - 4s 992us/step - loss: 22.0319 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+00 - val_loss: 22.0627 - val_acc: 0.0000e+00 - val_custom_accuracy: 0.0000e+00

Epoch 00032: val_loss did not improve from 22.05521
Epoch 33/10000
4500/4500 [==============================] - 4s 993us/step - loss: 22.0318 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+00 - val_lo


Epoch 00060: val_loss did not improve from 22.05521
Epoch 61/10000
4500/4500 [==============================] - 4s 992us/step - loss: 22.0312 - acc: 0.0062 - custom_accuracy: 0.0000e+00 - val_loss: 22.0674 - val_acc: 0.0000e+00 - val_custom_accuracy: 0.0000e+00

Epoch 00061: val_loss did not improve from 22.05521
Epoch 62/10000
4500/4500 [==============================] - 5s 1ms/step - loss: 22.0312 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+00 - val_loss: 22.0675 - val_acc: 0.0200 - val_custom_accuracy: 0.0000e+00

Epoch 00062: val_loss did not improve from 22.05521
Epoch 63/10000
4500/4500 [==============================] - 4s 996us/step - loss: 22.0311 - acc: 0.0184 - custom_accuracy: 0.0000e+00 - val_loss: 22.0676 - val_acc: 0.0000e+00 - val_custom_accuracy: 0.0000e+00

Epoch 00063: val_loss did not improve from 22.05521
Epoch 64/10000
4500/4500 [==============================] - 5s 1ms/step - loss: 22.0312 - acc: 0.0087 - custom_accuracy: 0.0000e+00 - val_loss: 22.0676 - val_ac

KeyboardInterrupt: 

In [21]:
model.fit(X, y, epochs= 1000,callbacks=[checkpointer], validation_split=0.1)

Train on 4500 samples, validate on 500 samples
Epoch 1/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 17.7344 - acc: 0.3184 - custom_accuracy: 0.0056 - val_loss: 19.4578 - val_acc: 0.2620 - val_custom_accuracy: 0.0020

Epoch 00001: val_acc improved from 0.26000 to 0.26200, saving model to model/model2/weights.01--19.46-0.2620.hdf5
Epoch 2/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 17.6716 - acc: 0.3204 - custom_accuracy: 0.0073 - val_loss: 19.4507 - val_acc: 0.2800 - val_custom_accuracy: 0.0000e+00

Epoch 00002: val_acc improved from 0.26200 to 0.28000, saving model to model/model2/weights.02--19.45-0.2800.hdf5
Epoch 3/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 17.6059 - acc: 0.3276 - custom_accuracy: 0.0051 - val_loss: 19.4327 - val_acc: 0.2560 - val_custom_accuracy: 0.0040

Epoch 00003: val_acc did not improve from 0.28000
Epoch 4/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 17.5803 


Epoch 00031: val_acc did not improve from 0.32400
Epoch 32/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 16.3039 - acc: 0.4018 - custom_accuracy: 0.0256 - val_loss: 18.6733 - val_acc: 0.3280 - val_custom_accuracy: 0.0020

Epoch 00032: val_acc improved from 0.32400 to 0.32800, saving model to model/model2/weights.32--18.67-0.3280.hdf5
Epoch 33/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 16.2643 - acc: 0.4058 - custom_accuracy: 0.0309 - val_loss: 18.6190 - val_acc: 0.3120 - val_custom_accuracy: 0.0000e+00

Epoch 00033: val_acc did not improve from 0.32800
Epoch 34/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 16.1979 - acc: 0.4147 - custom_accuracy: 0.0284 - val_loss: 18.5525 - val_acc: 0.3340 - val_custom_accuracy: 0.0020

Epoch 00034: val_acc improved from 0.32800 to 0.33400, saving model to model/model2/weights.34--18.55-0.3340.hdf5
Epoch 35/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 

4500/4500 [==============================] - 5s 1ms/step - loss: 13.7867 - acc: 0.4851 - custom_accuracy: 0.1187 - val_loss: 16.9840 - val_acc: 0.4040 - val_custom_accuracy: 0.0160

Epoch 00095: val_acc did not improve from 0.41400
Epoch 96/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 13.7161 - acc: 0.4798 - custom_accuracy: 0.1307 - val_loss: 17.0672 - val_acc: 0.4000 - val_custom_accuracy: 0.0080

Epoch 00096: val_acc did not improve from 0.41400
Epoch 97/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 13.6799 - acc: 0.4851 - custom_accuracy: 0.1364 - val_loss: 17.0859 - val_acc: 0.4220 - val_custom_accuracy: 0.0100

Epoch 00097: val_acc improved from 0.41400 to 0.42200, saving model to model/model2/weights.97--17.09-0.4220.hdf5
Epoch 98/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 13.6197 - acc: 0.4740 - custom_accuracy: 0.1376 - val_loss: 17.0069 - val_acc: 0.4100 - val_custom_accuracy: 0.0160

Epoch 00098: val

4500/4500 [==============================] - 5s 1ms/step - loss: 12.5870 - acc: 0.4996 - custom_accuracy: 0.2138 - val_loss: 16.2522 - val_acc: 0.4320 - val_custom_accuracy: 0.0240

Epoch 00127: val_acc did not improve from 0.44200
Epoch 128/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 12.5457 - acc: 0.4962 - custom_accuracy: 0.2118 - val_loss: 16.2303 - val_acc: 0.4480 - val_custom_accuracy: 0.0200

Epoch 00128: val_acc improved from 0.44200 to 0.44800, saving model to model/model2/weights.128--16.23-0.4480.hdf5
Epoch 129/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 12.5547 - acc: 0.5000 - custom_accuracy: 0.2102 - val_loss: 16.3295 - val_acc: 0.4180 - val_custom_accuracy: 0.0200

Epoch 00129: val_acc did not improve from 0.44800
Epoch 130/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 12.5398 - acc: 0.5031 - custom_accuracy: 0.2082 - val_loss: 16.1558 - val_acc: 0.4340 - val_custom_accuracy: 0.0160

Epoch 00130:


Epoch 00159: val_acc did not improve from 0.45600
Epoch 160/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 11.5277 - acc: 0.4964 - custom_accuracy: 0.3056 - val_loss: 15.4951 - val_acc: 0.4440 - val_custom_accuracy: 0.0300

Epoch 00160: val_acc did not improve from 0.45600
Epoch 161/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 11.5438 - acc: 0.4993 - custom_accuracy: 0.3009 - val_loss: 15.7740 - val_acc: 0.4380 - val_custom_accuracy: 0.0280

Epoch 00161: val_acc did not improve from 0.45600
Epoch 162/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 11.4706 - acc: 0.4956 - custom_accuracy: 0.3198 - val_loss: 15.4548 - val_acc: 0.4480 - val_custom_accuracy: 0.0280

Epoch 00162: val_acc did not improve from 0.45600
Epoch 163/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 11.4624 - acc: 0.4918 - custom_accuracy: 0.3180 - val_loss: 15.5909 - val_acc: 0.4580 - val_custom_accuracy: 0.0360

Epoch 00163

...

4500/4500 [==============================] - 5s 1ms/step - loss: 6.0241 - acc: 0.3542 - custom_accuracy: 0.9587 - val_loss: 12.2011 - val_acc: 0.4320 - val_custom_accuracy: 0.1080

Epoch 00687: val_acc did not improve from 0.46600
Epoch 688/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 6.0210 - acc: 0.3542 - custom_accuracy: 0.9560 - val_loss: 12.2776 - val_acc: 0.4260 - val_custom_accuracy: 0.0940

Epoch 00688: val_acc did not improve from 0.46600
Epoch 689/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 6.0190 - acc: 0.3524 - custom_accuracy: 0.9620 - val_loss: 12.1930 - val_acc: 0.4480 - val_custom_accuracy: 0.1200

Epoch 00689: val_acc did not improve from 0.46600
Epoch 690/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 6.0110 - acc: 0.3562 - custom_accuracy: 0.9587 - val_loss: 12.2543 - val_acc: 0.4400 - val_custom_accuracy: 0.0980

Epoch 00690: val_acc did not improve from 0.46600
Epoch 691/1000
4500/4500 [=====

4500/4500 [==============================] - 5s 1ms/step - loss: 5.9344 - acc: 0.3431 - custom_accuracy: 0.9680 - val_loss: 12.1165 - val_acc: 0.4220 - val_custom_accuracy: 0.1020

Epoch 00753: val_acc did not improve from 0.46600
Epoch 754/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 5.9489 - acc: 0.3331 - custom_accuracy: 0.9620 - val_loss: 12.3244 - val_acc: 0.4360 - val_custom_accuracy: 0.1060

Epoch 00754: val_acc did not improve from 0.46600
Epoch 755/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 5.9438 - acc: 0.3616 - custom_accuracy: 0.9649 - val_loss: 12.3224 - val_acc: 0.4340 - val_custom_accuracy: 0.0920

Epoch 00755: val_acc did not improve from 0.46600
Epoch 756/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 5.9356 - acc: 0.3442 - custom_accuracy: 0.9631 - val_loss: 12.2430 - val_acc: 0.4220 - val_custom_accuracy: 0.1020

Epoch 00756: val_acc did not improve from 0.46600
Epoch 757/1000
4500/4500 [=====

4500/4500 [==============================] - 5s 1ms/step - loss: 5.8615 - acc: 0.3538 - custom_accuracy: 0.9749 - val_loss: 12.2276 - val_acc: 0.4360 - val_custom_accuracy: 0.1080

Epoch 00819: val_acc did not improve from 0.46800
Epoch 820/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 5.8654 - acc: 0.3402 - custom_accuracy: 0.9716 - val_loss: 12.2097 - val_acc: 0.4220 - val_custom_accuracy: 0.0960

Epoch 00820: val_acc did not improve from 0.46800
Epoch 821/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 5.8747 - acc: 0.3518 - custom_accuracy: 0.9722 - val_loss: 12.0788 - val_acc: 0.4400 - val_custom_accuracy: 0.1160

Epoch 00821: val_acc did not improve from 0.46800
Epoch 822/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 5.8765 - acc: 0.3487 - custom_accuracy: 0.9704 - val_loss: 12.1591 - val_acc: 0.4400 - val_custom_accuracy: 0.1100

Epoch 00822: val_acc did not improve from 0.46800
Epoch 823/1000
4500/4500 [=====

4500/4500 [==============================] - 5s 1ms/step - loss: 5.8294 - acc: 0.3320 - custom_accuracy: 0.9744 - val_loss: 12.0104 - val_acc: 0.4480 - val_custom_accuracy: 0.1200

Epoch 00885: val_acc did not improve from 0.46800
Epoch 886/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 5.8163 - acc: 0.3338 - custom_accuracy: 0.9793 - val_loss: 12.0788 - val_acc: 0.4200 - val_custom_accuracy: 0.1160

Epoch 00886: val_acc did not improve from 0.46800
Epoch 887/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 5.8272 - acc: 0.3300 - custom_accuracy: 0.9724 - val_loss: 12.0855 - val_acc: 0.4180 - val_custom_accuracy: 0.1160

Epoch 00887: val_acc did not improve from 0.46800
Epoch 888/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 5.8289 - acc: 0.3471 - custom_accuracy: 0.9744 - val_loss: 12.1471 - val_acc: 0.4100 - val_custom_accuracy: 0.1000

Epoch 00888: val_acc did not improve from 0.46800
Epoch 889/1000
4500/4500 [=====

4500/4500 [==============================] - 5s 1ms/step - loss: 5.7911 - acc: 0.3462 - custom_accuracy: 0.9771 - val_loss: 12.0214 - val_acc: 0.4420 - val_custom_accuracy: 0.1240

Epoch 00951: val_acc did not improve from 0.46800
Epoch 952/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 5.7947 - acc: 0.3376 - custom_accuracy: 0.9747 - val_loss: 12.0193 - val_acc: 0.4460 - val_custom_accuracy: 0.1060

Epoch 00952: val_acc did not improve from 0.46800
Epoch 953/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 5.7924 - acc: 0.3347 - custom_accuracy: 0.9729 - val_loss: 12.1973 - val_acc: 0.4580 - val_custom_accuracy: 0.0980

Epoch 00953: val_acc did not improve from 0.46800
Epoch 954/1000
4500/4500 [==============================] - 5s 1ms/step - loss: 5.7852 - acc: 0.3307 - custom_accuracy: 0.9767 - val_loss: 12.1157 - val_acc: 0.4140 - val_custom_accuracy: 0.1040

Epoch 00954: val_acc did not improve from 0.46800
Epoch 955/1000
4500/4500 [=====

In [22]:
#保存权重和模型
model.save_weights('model/model2/captcha_model_weights.h5')
model.save('model/model2/captcha__model_2.h5')

# 模型二

In [19]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np
import h5py
import os
 
from keras.utils import np_utils, conv_utils
from keras.models import Model
from keras.layers import Flatten, Dense, Input 
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50
from keras import backend as K
from keras.callbacks import *

In [38]:
# 图片的长度和宽度
width = 120
height = 40

input_tensor = Input( shape=(height, width, 3))
base_model = ResNet50(input_tensor=input_tensor,include_top=False,weights='imagenet')
x = base_model.layers[-2].output
print(x.shape)
x = Flatten()(x)
#为输入数据施加Dropout。Dropout将在训练过程中每次更新参数时随机断开一定百分比（rate）的输入神经元，Dropout层用于防止过拟合。
x = Dropout(0.25)(x)
x = Dense(1024,activation='relu')(x)
x= BatchNormalization()(x)
#Dense就是常用的全连接层
#最后连接4个分类器，每个分类器是56个神经元，分别输出56个字符的概率。
x = [Dense(word_class, activation='softmax', name='c%d'%(i+1))(x) for i in range(word_len)]
# x = [Dense(word_class, activation='sigmoid', name='c%d'%(i+1))(x) for i in range(word_len)]
output = concatenate(x)
#构建模型
model = Model(inputs=input_tensor, outputs=output)
adam = Adam(lr=0.1)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy',custom_accuracy])

(?, 2, 4, 2048)


In [39]:

#每次epoch都保存一下权重，用于继续训练
early_stopping = EarlyStopping(monitor='val_loss', patience=50)
#plateau = ReduceLROnPlateau(monitor="val_acc", verbose=1, mode='max', factor=0.5, patience=2)
plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.00001, cooldown=0, min_lr=0)

checkpointer = ModelCheckpoint(filepath="model/model1/ResNet50_weights.{epoch:02d}--{val_loss:.2f}-{val_acc:.4f}.hdf5", monitor='val_loss', 
                                 verbose=2, save_best_only=True, mode='min')
#开始训练，validation_split代表10%的数据不参与训练，用于做验证急
#我之前训练了50个epochs以上，这里根据自己的情况进行选择。如果输出的val_acc已经达到你满意的数值，可以终止训练
model.fit(X, y, epochs= 100000,callbacks=[checkpointer,plateau], validation_split=0.1)
# plot model
plot_model(model, to_file='model/model1/ResNet50_model.png', show_shapes=True)

#保存权重和模型
model.save_weights('model/model1/ResNet50_captcha_model_weights.h5')
model.save('model/model1/ResNet50_captcha__model_2.h5')

Train on 4500 samples, validate on 500 samples
Epoch 1/100
4500/4500 [==============================] - 38s 8ms/step - loss: 61.3549 - acc: 0.0091 - custom_accuracy: 0.0000e+00 - val_loss: 63.2349 - val_acc: 0.0140 - val_custom_accuracy: 0.0000e+00

Epoch 00001: val_loss improved from inf to 63.23491, saving model to model/model1/ResNet50_weights.01--63.23-0.0140.hdf5
Epoch 2/100
4500/4500 [==============================] - 24s 5ms/step - loss: 63.5489 - acc: 0.0162 - custom_accuracy: 0.0000e+00 - val_loss: 63.3528 - val_acc: 0.0180 - val_custom_accuracy: 0.0000e+00

Epoch 00002: val_loss did not improve from 63.23491
Epoch 3/100
4500/4500 [==============================] - 24s 5ms/step - loss: 63.5787 - acc: 0.0162 - custom_accuracy: 0.0000e+00 - val_loss: 63.5885 - val_acc: 0.0120 - val_custom_accuracy: 0.0000e+00

Epoch 00003: val_loss did not improve from 63.23491
Epoch 4/100
4500/4500 [==============================] - 24s 5ms/step - loss: 63.5638 - acc: 0.0147 - custom_accuracy: 


Epoch 00032: val_loss did not improve from 63.23491
Epoch 33/100
4500/4500 [==============================] - 24s 5ms/step - loss: 63.5917 - acc: 0.0158 - custom_accuracy: 0.0000e+00 - val_loss: 63.4706 - val_acc: 0.0140 - val_custom_accuracy: 0.0000e+00

Epoch 00033: val_loss did not improve from 63.23491
Epoch 34/100
4500/4500 [==============================] - 24s 5ms/step - loss: 63.5950 - acc: 0.0151 - custom_accuracy: 0.0000e+00 - val_loss: 63.4706 - val_acc: 0.0140 - val_custom_accuracy: 0.0000e+00

Epoch 00034: val_loss did not improve from 63.23491
Epoch 35/100
4500/4500 [==============================] - 24s 5ms/step - loss: 63.5885 - acc: 0.0151 - custom_accuracy: 0.0000e+00 - val_loss: 63.4706 - val_acc: 0.0140 - val_custom_accuracy: 0.0000e+00

Epoch 00035: val_loss did not improve from 63.23491
Epoch 36/100
4500/4500 [==============================] - 24s 5ms/step - loss: 63.5950 - acc: 0.0156 - custom_accuracy: 0.0000e+00 - val_loss: 63.5001 - val_acc: 0.0140 - val_cust

KeyboardInterrupt: 

In [ ]:
model.fit(X, y, epochs= 100000,callbacks=[checkpointer,plateau], validation_split=0.1)

In [34]:
model.layers[-2].output

<tf.Tensor 'avg_pool/Mean:0' shape=(?, 2048) dtype=float32>

# 模型三

In [50]:
#创建输入，结构为 高，宽，通道
input_tensor = Input( shape=(height, width, 3))

x = input_tensor

#构建卷积网络
#两层卷积层，一层池化层，重复3次。因为生成的验证码比较小，padding使用same
x = Convolution2D(64, 5, padding='same', activation='relu')(x)
x = Dropout(0.25)(x)
x = Convolution2D(64, 3, padding='same', activation='relu')(x)
x = Dropout(0.25)(x)
x= BatchNormalization()(x)
#x = MaxPooling2D((2, 2))(x)
x = Dropout(0.25)(x)


x = Convolution2D(128, 3, padding='same', activation='relu')(x)
x = Dropout(0.25)(x)
x = Convolution2D(128, 3, padding='same',activation='relu')(x)
x = Dropout(0.25)(x)
x= BatchNormalization()(x)
#x = MaxPooling2D((2, 2))(x)

#Flatten层用来将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡。
x = Flatten()(x)
#为输入数据施加Dropout。Dropout将在训练过程中每次更新参数时随机断开一定百分比（rate）的输入神经元，Dropout层用于防止过拟合。
x = Dropout(0.25)(x)
x= BatchNormalization()(x)
#Dense就是常用的全连接层
#最后连接4个分类器，每个分类器是56个神经元，分别输出56个字符的概率。
x = [Dense(word_class, activation='sigmoid', name='c%d'%(i+1))(x) for i in range(word_len)]
# x = [Dense(word_class, activation='sigmoid', name='c%d'%(i+1))(x) for i in range(word_len)]
output = concatenate(x)
#构建模型
model = Model(inputs=input_tensor, outputs=output)
# model.add(BatchNormalization())
#因为训练可能需要数个小时，所以这里加载了之前我训练好的参数。准确率为94%
#可以直接使用此参数继续进行训练，也可以自己从头开始训练
# model.load_weights('model/weights.10--6.55-0.3062.hdf5')

#这里优化器选用Adadelta，学习率0.1
opt = Adadelta(lr=0.01)
#编译模型以供训练，损失函数使用 categorical_crossentropy，使用accuracy评估模型在训练和测试时的性能的指标
#这里使用自定义的评估模型
model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics=['accuracy',custom_accuracy])
#每次epoch都保存一下权重，用于继续训练
checkpointer = ModelCheckpoint(filepath="model/weights.{epoch:02d}--{val_loss:.2f}-{val_acc:.4f}.hdf5",
                               verbose=2, save_weights_only=True)
#开始训练，validation_split代表10%的数据不参与训练，用于做验证急
#我之前训练了50个epochs以上，这里根据自己的情况进行选择。如果输出的val_acc已经达到你满意的数值，可以终止训练
model.fit(X, y, epochs= 1000,callbacks=[checkpointer], validation_split=0.1)
# plot model
plot_model(model, to_file='model/model1.png', show_shapes=True)

#保存权重和模型
model.save_weights('model/captcha_model_weights1.h5')
model.save('model/captcha__model_21.h5')


Train on 4500 samples, validate on 500 samples
Epoch 1/1000
3744/4500 [=======================>......] - ETA: 3:37 - loss: 22.9677 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+ - ETA: 1:53 - loss: 22.8143 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+ - ETA: 1:19 - loss: 22.9169 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+ - ETA: 1:02 - loss: 22.8610 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+ - ETA: 51s - loss: 22.8279 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+00 - ETA: 44s - loss: 22.9309 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 39s - loss: 22.8861 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 36s - loss: 22.8851 - acc: 0.0039 - custom_accuracy: 0.0000e+00    - ETA: 33s - loss: 22.9324 - acc: 0.0035 - custom_accuracy: 0.0000e+0 - ETA: 30s - loss: 22.9596 - acc: 0.0031 - custom_accuracy: 0.0000e+0 - ETA: 28s - loss: 22.9399 - acc: 0.0057 - custom_accuracy: 0.0000e+0 - ETA: 27s - loss: 22.9380 - acc: 0.0052 - custom_accuracy: 0.0000e+0 - ETA: 25s - loss: 22.93

3808/4500 [========================>.....] - ETA: 11s - loss: 22.8009 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.7660 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.6987 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.7388 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.6875 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.6254 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.6858 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.6733 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.6771 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.7148 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.7230 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.7359 - acc: 0.0026 - custom_accuracy: 0.0000e+00    - ETA: 11s - loss: 22.6968 - acc: 0.0024 - custom_accuracy: 0.0000e+0

4500/4500 [==============================] - ETA: 1s - loss: 22.7432 - acc: 0.0060 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.7446 - acc: 0.0059 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.7420 - acc: 0.0059 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.7408 - acc: 0.0058 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.7401 - acc: 0.0058 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.7414 - acc: 0.0057 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.7413 - acc: 0.0057 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.7439 - acc: 0.0057 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.7455 - acc: 0.0056 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.7436 - acc: 0.0056 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.7420 - acc: 0.0055 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.7431 - acc: 0.0055 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.7420 - acc: 0.0054 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.7429 - acc: 0.0054 - custom_accuracy: 0.0000e+ - ETA

3808/4500 [========================>.....] - ETA: 12s - loss: 23.0919 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 12s - loss: 22.8199 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 12s - loss: 22.7401 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 12s - loss: 22.7636 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 12s - loss: 22.7103 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.6639 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.6730 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.6712 - acc: 0.0039 - custom_accuracy: 0.0000e+00    - ETA: 11s - loss: 22.7175 - acc: 0.0035 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.6779 - acc: 0.0063 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.6655 - acc: 0.0085 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.7078 - acc: 0.0078 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.6897 - acc: 0.0072 - custom_accuracy: 0.0000e+0 - ETA: 11s - lo

4500/4500 [==============================] - ETA: 1s - loss: 22.6679 - acc: 0.0057 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.6666 - acc: 0.0057 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.6671 - acc: 0.0056 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.6664 - acc: 0.0056 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.6650 - acc: 0.0055 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.6691 - acc: 0.0055 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.6723 - acc: 0.0055 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.6772 - acc: 0.0054 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.6782 - acc: 0.0054 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.6772 - acc: 0.0053 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.6800 - acc: 0.0053 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.6770 - acc: 0.0052 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.6743 - acc: 0.0054 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.6752 - acc: 0.0054 - custom_accuracy: 0.0000e+ - ETA

3808/4500 [========================>.....] - ETA: 11s - loss: 22.4838 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 12s - loss: 22.6316 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 12s - loss: 22.5602 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 12s - loss: 22.6150 - acc: 0.0078 - custom_accuracy: 0.0000e+00    - ETA: 11s - loss: 22.5831 - acc: 0.0063 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.5449 - acc: 0.0104 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.5589 - acc: 0.0089 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.5857 - acc: 0.0078 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.5948 - acc: 0.0069 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.5738 - acc: 0.0063 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.5599 - acc: 0.0057 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.5330 - acc: 0.0052 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.4772 - acc: 0.0072 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.4559 - ac

4500/4500 [==============================] - ETA: 1s - loss: 22.5670 - acc: 0.0065 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.5666 - acc: 0.0065 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.5657 - acc: 0.0064 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.5682 - acc: 0.0064 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.5673 - acc: 0.0063 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.5633 - acc: 0.0063 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.5618 - acc: 0.0062 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.5604 - acc: 0.0062 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.5604 - acc: 0.0061 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.5596 - acc: 0.0061 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.5599 - acc: 0.0063 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.5626 - acc: 0.0062 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.5618 - acc: 0.0062 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.5634 - acc: 0.0063 - custom_accuracy: 0.0000e+ - ETA

3808/4500 [========================>.....] - ETA: 11s - loss: 22.0789 - acc: 0.0000e+00 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.1994 - acc: 0.0156 - custom_accuracy: 0.0000e+00    - ETA: 11s - loss: 22.1568 - acc: 0.0104 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.2033 - acc: 0.0078 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.1440 - acc: 0.0063 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.2677 - acc: 0.0052 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.2660 - acc: 0.0045 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.2923 - acc: 0.0039 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.3502 - acc: 0.0069 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.3686 - acc: 0.0063 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.3574 - acc: 0.0057 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.3786 - acc: 0.0052 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.3717 - acc: 0.0048 - custom_accuracy: 0.0000e+0 - ETA: 11s - loss: 22.3426 - acc: 0.006

4500/4500 [==============================] - ETA: 1s - loss: 22.4194 - acc: 0.0068 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.4224 - acc: 0.0070 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.4196 - acc: 0.0072 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.4195 - acc: 0.0074 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.4181 - acc: 0.0076 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.4207 - acc: 0.0080 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.4234 - acc: 0.0079 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.4247 - acc: 0.0081 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.4243 - acc: 0.0081 - custom_accuracy: 0.0000e+ - ETA: 1s - loss: 22.4285 - acc: 0.0080 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.4270 - acc: 0.0079 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.4232 - acc: 0.0081 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.4225 - acc: 0.0080 - custom_accuracy: 0.0000e+ - ETA: 0s - loss: 22.4241 - acc: 0.0080 - custom_accuracy: 0.0000e+ - ETA

KeyboardInterrupt: 

# 模型四

## 数据处理

In [11]:
from tqdm import tqdm
import scipy
from scipy import misc
import numpy as np
import os
from keras.utils import to_categorical
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint

In [12]:

labels = ['0','1','2','3','4','5','6','7','8','9',
           'A','B','C','D','E','F','G','H','I','J',
           'K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
           'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p',
           'q','r','s','t','u','v','w','x','y','z'
           ]
label2id = {} # 标签转id
id2label = {} # id转标签

for label in labels:
    label2id[label] = len(label2id)
for label, id in label2id.items():
    id2label[id] = label

In [13]:
# 数据集
labels1 = []
labels2 = []
labels3 = []
labels4 = []
print('PREPARING...')
for strs in tqdm(image_label):
    l1, l2, l3, l4 = tuple(strs[:4]) # 切分出验证码的四位标签
    labels1.append(label2id[l1])
    labels2.append(label2id[l2])
    labels3.append(label2id[l3])
    labels4.append(label2id[l4])

PREPARING...


100%|██████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 996603.15it/s]


In [14]:

# 按9:1划分训练集和验证集
train_data = [j for i, j in enumerate(X) if i % 10 != 0]
train_labels1 = [j for i, j in enumerate(labels1) if i % 10 != 0]
train_labels2 = [j for i, j in enumerate(labels2) if i % 10 != 0]
train_labels3 = [j for i, j in enumerate(labels3) if i % 10 != 0]
train_labels4 = [j for i, j in enumerate(labels4) if i % 10 != 0]
dev_data = [j for i, j in enumerate(X) if i % 10 == 0]
dev_labels1 = [j for i, j in enumerate(labels1) if i % 10 == 0]
dev_labels2 = [j for i, j in enumerate(labels2) if i % 10 == 0]
dev_labels3 = [j for i, j in enumerate(labels3) if i % 10 == 0]
dev_labels4 = [j for i, j in enumerate(labels4) if i % 10 == 0]

# 转换为 numpy 对象
train_data = np.asarray(train_data)
train_labels1 = to_categorical(np.asarray(train_labels1), num_classes=62)
train_labels2 = to_categorical(np.asarray(train_labels2), num_classes=62)
train_labels3 = to_categorical(np.asarray(train_labels3), num_classes=62)
train_labels4 = to_categorical(np.asarray(train_labels4), num_classes=62)
dev_data = np.asarray(dev_data)
dev_labels1 = to_categorical(np.asarray(dev_labels1), num_classes=62)
dev_labels2 = to_categorical(np.asarray(dev_labels2), num_classes=62)
dev_labels3 = to_categorical(np.asarray(dev_labels3), num_classes=62)
dev_labels4 = to_categorical(np.asarray(dev_labels4), num_classes=62)

## 数据处理

In [56]:
from tqdm import tqdm
import scipy
from scipy import misc
import numpy as np
import os
from keras.utils import to_categorical
from keras.layers import *
from keras.models import Model
from keras.callbacks import ModelCheckpoint

In [60]:
labels = ['0','1','2','3','4','5','6','7','8','9',
           'A','B','C','D','E','F','G','H','I','J',
           'K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
           'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p',
           'q','r','s','t','u','v','w','x','y','z'
           ]
label2id = {} # 标签转id
id2label = {} # id转标签

for label in labels:
    label2id[label] = len(label2id)
for label, id in label2id.items():
    id2label[id] = label

In [61]:
# 数据集
labels1 = []
labels2 = []
labels3 = []
labels4 = []
print('PREPARING...')
for strs in tqdm(image_labels):
    l1, l2, l3, l4 = tuple(strs[:4]) # 切分出验证码的四位标签
    labels1.append(label2id[l1])
    labels2.append(label2id[l2])
    labels3.append(label2id[l3])
    labels4.append(label2id[l4])

PREPARING...


100%|███████████████████████████████████████████████████████████████████████| 25000/25000 [00:00<00:00, 1089882.55it/s]


In [ ]:
train_datas = np.asarray(X)
train_labels1s = to_categorical(np.asarray(labels1), num_classes=62)
train_labels2s = to_categorical(np.asarray(labels2), num_classes=62)
train_labels3s = to_categorical(np.asarray(labels3), num_classes=62)
train_labels4s = to_categorical(np.asarray(labels4), num_classes=62)

In [ ]:
from sklearn.model_selection import train_test_split
train_data,train_labels1,train_labels2,train_labels3,train_labels4,dev_data, dev_labels1, dev_labels2, dev_labels3, dev_labels4= train_test_split(train_datas,train_labels1s,train_labels2s,train_labels3s,train_labels4s,test_size=0.2,random_state=20190)

In [64]:
# 按9:1划分训练集和验证集
train_data = [j for i, j in enumerate(X) if i % 9 != 0]
train_labels1 = [j for i, j in enumerate(labels1) if i % 9 != 0]
train_labels2 = [j for i, j in enumerate(labels2) if i % 9 != 0]
train_labels3 = [j for i, j in enumerate(labels3) if i % 9 != 0]
train_labels4 = [j for i, j in enumerate(labels4) if i % 9 != 0]
dev_data = [j for i, j in enumerate(X) if i % 9 == 0]
dev_labels1 = [j for i, j in enumerate(labels1) if i % 9 == 0]
dev_labels2 = [j for i, j in enumerate(labels2) if i % 9 == 0]
dev_labels3 = [j for i, j in enumerate(labels3) if i % 9 == 0]
dev_labels4 = [j for i, j in enumerate(labels4) if i % 9 == 0]

# 转换为 numpy 对象
train_data = np.asarray(train_data)
train_labels1 = to_categorical(np.asarray(train_labels1), num_classes=62)
train_labels2 = to_categorical(np.asarray(train_labels2), num_classes=62)
train_labels3 = to_categorical(np.asarray(train_labels3), num_classes=62)
train_labels4 = to_categorical(np.asarray(train_labels4), num_classes=62)
dev_data = np.asarray(dev_data)
dev_labels1 = to_categorical(np.asarray(dev_labels1), num_classes=62)
dev_labels2 = to_categorical(np.asarray(dev_labels2), num_classes=62)
dev_labels3 = to_categorical(np.asarray(dev_labels3), num_classes=62)
dev_labels4 = to_categorical(np.asarray(dev_labels4), num_classes=62)

## 模型0 92

In [16]:
pic_in = Input( shape=(height, width, 3))
# Block 1
cnn_features = Conv2D(32, (3,3), activation='relu', padding='same')(pic_in)
cnn_features = Conv2D(32, (3,3), activation='relu')(cnn_features)
cnn_features = MaxPooling2D((2, 2))(cnn_features)
cnn_features = Dropout(0.25)(cnn_features)
# Block 2
cnn_features = Conv2D(64, (3,3), activation='relu',padding='same')(cnn_features)
cnn_features = Conv2D(64, (3,3), activation='relu')(cnn_features)
cnn_features = MaxPooling2D((2, 2))(cnn_features)
cnn_features = Dropout(0.25)(cnn_features)
# Block 3
cnn_features = Conv2D(128, (3,3), activation='relu',padding='same')(cnn_features)
cnn_features = Conv2D(128, (3,3), activation='relu')(cnn_features)
cnn_features = MaxPooling2D((2, 2))(cnn_features)
cnn_features = Dropout(0.25)(cnn_features)
cnn_features = Flatten()(cnn_features)
# classifier 1
output_l1 = Dense(128, activation='relu')(cnn_features)
output_l1 = Dropout(0.5)(output_l1)
output_l1 = Dense(word_class, activation='sigmoid')(output_l1)
# classifier 2
output_l2 = Dense(128, activation='relu')(cnn_features)
output_l2 = Dropout(0.5)(output_l2)
output_l2 = Dense(word_class, activation='sigmoid')(output_l2)
# classifier 3
output_l3 = Dense(128, activation='relu')(cnn_features)
output_l3 = Dropout(0.5)(output_l3)
output_l3 = Dense(word_class, activation='sigmoid')(output_l3)
# classifier 4
output_l4 = Dense(128, activation='relu')(cnn_features)
output_l4 = Dropout(0.5)(output_l4)
output_l4 = Dense(word_class, activation='sigmoid')(output_l4)

model = Model(inputs=pic_in, outputs=[output_l1, output_l2, output_l3, output_l4])

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 40, 120, 3)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 40, 120, 32)  896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 38, 118, 32)  9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 19, 59, 32)   0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
dropout_1 

In [18]:
from keras.models import load_model
model = load_model('model_942.hdf5')

In [19]:
plot_model(model, to_file='model/models.png', show_shapes=True)

In [36]:
keras.optimizers.RMSprop()

In [37]:
adam = Adam(lr=0.00001)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              loss_weights=[0.8, 0.85, 0.9, 0.95], # 给四个分类器同样的权重
              metrics=['accuracy'])

print('TRAINING...')
plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20, verbose=1, mode='auto')
#plateau = ReduceLROnPlateau(monitor="val_loss", verbose=1, mode='min', factor=0.5, patience=2)
checkpoint = ModelCheckpoint('model.hdf5', monitor='val_loss', 
                                 verbose=2, save_best_only=True, mode='min')
keras.optimizers.SGD(lr=0.0000001, momentum=0.0001, decay=0.00001, nesterov=True)
model.fit(train_data, [train_labels1, train_labels2, train_labels3, train_labels4],
          epochs=100,
          batch_size=128,
          validation_data=(dev_data, [dev_labels1, dev_labels2, dev_labels3, dev_labels4]),
          callbacks=[checkpoint,plateau], verbose=1, shuffle=True)

TRAINING...
Train on 4500 samples, validate on 500 samples
Epoch 1/100
4500/4500 [==============================] - ETA: 1:11 - loss: 0.3720 - dense_2_loss: 0.0412 - dense_4_loss: 0.1906 - dense_6_loss: 0.0786 - dense_8_loss: 0.1119 - dense_2_acc: 0.9844 - dense_4_acc: 0.9453 - dense_6_acc: 0.9688 - dense_8_acc: 0.968 - ETA: 36s - loss: 0.2964 - dense_2_loss: 0.0355 - dense_4_loss: 0.1190 - dense_6_loss: 0.0775 - dense_8_loss: 0.1022 - dense_2_acc: 0.9805 - dense_4_acc: 0.9609 - dense_6_acc: 0.9688 - dense_8_acc: 0.964 - ETA: 24s - loss: 0.2457 - dense_2_loss: 0.0437 - dense_4_loss: 0.1054 - dense_6_loss: 0.0557 - dense_8_loss: 0.0747 - dense_2_acc: 0.9766 - dense_4_acc: 0.9661 - dense_6_acc: 0.9792 - dense_8_acc: 0.97 - ETA: 18s - loss: 0.2162 - dense_2_loss: 0.0362 - dense_4_loss: 0.0905 - dense_6_loss: 0.0593 - dense_8_loss: 0.0598 - dense_2_acc: 0.9805 - dense_4_acc: 0.9707 - dense_6_acc: 0.9766 - dense_8_acc: 0.98 - ETA: 14s - loss: 0.2038 - dense_2_loss: 0.0375 - dense_4_loss: 0.

4500/4500 [==============================] - ETA: 2s - loss: 0.1324 - dense_2_loss: 0.0557 - dense_4_loss: 0.0751 - dense_6_loss: 0.0051 - dense_8_loss: 0.0205 - dense_2_acc: 0.9844 - dense_4_acc: 0.9688 - dense_6_acc: 1.0000 - dense_8_acc: 0.984 - ETA: 2s - loss: 0.1640 - dense_2_loss: 0.0424 - dense_4_loss: 0.0816 - dense_6_loss: 0.0346 - dense_8_loss: 0.0312 - dense_2_acc: 0.9883 - dense_4_acc: 0.9609 - dense_6_acc: 0.9922 - dense_8_acc: 0.984 - ETA: 2s - loss: 0.2027 - dense_2_loss: 0.0601 - dense_4_loss: 0.1067 - dense_6_loss: 0.0292 - dense_8_loss: 0.0396 - dense_2_acc: 0.9740 - dense_4_acc: 0.9531 - dense_6_acc: 0.9922 - dense_8_acc: 0.981 - ETA: 2s - loss: 0.1940 - dense_2_loss: 0.0551 - dense_4_loss: 0.0863 - dense_6_loss: 0.0499 - dense_8_loss: 0.0333 - dense_2_acc: 0.9766 - dense_4_acc: 0.9648 - dense_6_acc: 0.9883 - dense_8_acc: 0.986 - ETA: 2s - loss: 0.2117 - dense_2_loss: 0.0570 - dense_4_loss: 0.0782 - dense_6_loss: 0.0623 - dense_8_loss: 0.0458 - dense_2_acc: 0.9734 - 

4500/4500 [==============================] - ETA: 2s - loss: 0.1263 - dense_2_loss: 0.0347 - dense_4_loss: 0.0325 - dense_6_loss: 0.0101 - dense_8_loss: 0.0650 - dense_2_acc: 0.9844 - dense_4_acc: 0.9844 - dense_6_acc: 1.0000 - dense_8_acc: 0.968 - ETA: 2s - loss: 0.1621 - dense_2_loss: 0.0349 - dense_4_loss: 0.0382 - dense_6_loss: 0.0722 - dense_8_loss: 0.0386 - dense_2_acc: 0.9883 - dense_4_acc: 0.9844 - dense_6_acc: 0.9805 - dense_8_acc: 0.984 - ETA: 2s - loss: 0.1440 - dense_2_loss: 0.0266 - dense_4_loss: 0.0366 - dense_6_loss: 0.0598 - dense_8_loss: 0.0397 - dense_2_acc: 0.9896 - dense_4_acc: 0.9818 - dense_6_acc: 0.9844 - dense_8_acc: 0.984 - ETA: 2s - loss: 0.1438 - dense_2_loss: 0.0332 - dense_4_loss: 0.0371 - dense_6_loss: 0.0520 - dense_8_loss: 0.0410 - dense_2_acc: 0.9883 - dense_4_acc: 0.9824 - dense_6_acc: 0.9883 - dense_8_acc: 0.982 - ETA: 2s - loss: 0.1433 - dense_2_loss: 0.0347 - dense_4_loss: 0.0346 - dense_6_loss: 0.0535 - dense_8_loss: 0.0401 - dense_2_acc: 0.9875 - 

4500/4500 [==============================] - ETA: 2s - loss: 0.0462 - dense_2_loss: 0.0232 - dense_4_loss: 0.0176 - dense_6_loss: 0.0100 - dense_8_loss: 0.0040 - dense_2_acc: 0.9922 - dense_4_acc: 0.9922 - dense_6_acc: 1.0000 - dense_8_acc: 1.000 - ETA: 2s - loss: 0.1215 - dense_2_loss: 0.0399 - dense_4_loss: 0.0173 - dense_6_loss: 0.0516 - dense_8_loss: 0.0299 - dense_2_acc: 0.9844 - dense_4_acc: 0.9961 - dense_6_acc: 0.9883 - dense_8_acc: 0.984 - ETA: 2s - loss: 0.1263 - dense_2_loss: 0.0635 - dense_4_loss: 0.0163 - dense_6_loss: 0.0382 - dense_8_loss: 0.0287 - dense_2_acc: 0.9766 - dense_4_acc: 0.9974 - dense_6_acc: 0.9922 - dense_8_acc: 0.984 - ETA: 2s - loss: 0.1311 - dense_2_loss: 0.0577 - dense_4_loss: 0.0166 - dense_6_loss: 0.0492 - dense_8_loss: 0.0279 - dense_2_acc: 0.9785 - dense_4_acc: 0.9980 - dense_6_acc: 0.9863 - dense_8_acc: 0.986 - ETA: 2s - loss: 0.1354 - dense_2_loss: 0.0558 - dense_4_loss: 0.0205 - dense_6_loss: 0.0423 - dense_8_loss: 0.0372 - dense_2_acc: 0.9797 - 

4500/4500 [==============================] - ETA: 2s - loss: 0.1495 - dense_2_loss: 0.0320 - dense_4_loss: 0.0833 - dense_6_loss: 0.0528 - dense_8_loss: 0.0059 - dense_2_acc: 0.9844 - dense_4_acc: 0.9609 - dense_6_acc: 0.9844 - dense_8_acc: 1.000 - ETA: 2s - loss: 0.1545 - dense_2_loss: 0.0322 - dense_4_loss: 0.0789 - dense_6_loss: 0.0347 - dense_8_loss: 0.0321 - dense_2_acc: 0.9883 - dense_4_acc: 0.9648 - dense_6_acc: 0.9883 - dense_8_acc: 0.988 - ETA: 2s - loss: 0.1705 - dense_2_loss: 0.0443 - dense_4_loss: 0.0690 - dense_6_loss: 0.0303 - dense_8_loss: 0.0517 - dense_2_acc: 0.9870 - dense_4_acc: 0.9661 - dense_6_acc: 0.9922 - dense_8_acc: 0.984 - ETA: 2s - loss: 0.1786 - dense_2_loss: 0.0485 - dense_4_loss: 0.0645 - dense_6_loss: 0.0388 - dense_8_loss: 0.0527 - dense_2_acc: 0.9805 - dense_4_acc: 0.9707 - dense_6_acc: 0.9922 - dense_8_acc: 0.982 - ETA: 2s - loss: 0.1844 - dense_2_loss: 0.0566 - dense_4_loss: 0.0550 - dense_6_loss: 0.0451 - dense_8_loss: 0.0544 - dense_2_acc: 0.9812 - 

...

4500/4500 [==============================] - ETA: 2s - loss: 0.2540 - dense_2_loss: 0.0398 - dense_4_loss: 0.0951 - dense_6_loss: 0.0851 - dense_8_loss: 0.0681 - dense_2_acc: 0.9844 - dense_4_acc: 0.9453 - dense_6_acc: 0.9688 - dense_8_acc: 0.976 - ETA: 2s - loss: 0.1779 - dense_2_loss: 0.0408 - dense_4_loss: 0.0688 - dense_6_loss: 0.0441 - dense_8_loss: 0.0495 - dense_2_acc: 0.9883 - dense_4_acc: 0.9688 - dense_6_acc: 0.9844 - dense_8_acc: 0.984 - ETA: 2s - loss: 0.2122 - dense_2_loss: 0.0678 - dense_4_loss: 0.0882 - dense_6_loss: 0.0409 - dense_8_loss: 0.0486 - dense_2_acc: 0.9818 - dense_4_acc: 0.9661 - dense_6_acc: 0.9870 - dense_8_acc: 0.981 - ETA: 2s - loss: 0.2160 - dense_2_loss: 0.0702 - dense_4_loss: 0.0882 - dense_6_loss: 0.0379 - dense_8_loss: 0.0534 - dense_2_acc: 0.9824 - dense_4_acc: 0.9668 - dense_6_acc: 0.9883 - dense_8_acc: 0.982 - ETA: 2s - loss: 0.1926 - dense_2_loss: 0.0644 - dense_4_loss: 0.0800 - dense_6_loss: 0.0334 - dense_8_loss: 0.0452 - dense_2_acc: 0.9812 - 

4500/4500 [==============================] - ETA: 2s - loss: 0.1849 - dense_2_loss: 0.0456 - dense_4_loss: 0.0349 - dense_6_loss: 0.0907 - dense_8_loss: 0.0390 - dense_2_acc: 0.9844 - dense_4_acc: 0.9844 - dense_6_acc: 0.9531 - dense_8_acc: 0.984 - ETA: 2s - loss: 0.1694 - dense_2_loss: 0.0393 - dense_4_loss: 0.0319 - dense_6_loss: 0.0681 - dense_8_loss: 0.0521 - dense_2_acc: 0.9883 - dense_4_acc: 0.9844 - dense_6_acc: 0.9648 - dense_8_acc: 0.980 - ETA: 2s - loss: 0.1444 - dense_2_loss: 0.0377 - dense_4_loss: 0.0334 - dense_6_loss: 0.0481 - dense_8_loss: 0.0448 - dense_2_acc: 0.9870 - dense_4_acc: 0.9844 - dense_6_acc: 0.9766 - dense_8_acc: 0.984 - ETA: 2s - loss: 0.1476 - dense_2_loss: 0.0338 - dense_4_loss: 0.0502 - dense_6_loss: 0.0384 - dense_8_loss: 0.0456 - dense_2_acc: 0.9883 - dense_4_acc: 0.9844 - dense_6_acc: 0.9824 - dense_8_acc: 0.982 - ETA: 2s - loss: 0.1459 - dense_2_loss: 0.0354 - dense_4_loss: 0.0578 - dense_6_loss: 0.0348 - dense_8_loss: 0.0391 - dense_2_acc: 0.9875 - 

4500/4500 [==============================] - ETA: 2s - loss: 0.2882 - dense_2_loss: 0.0307 - dense_4_loss: 0.0888 - dense_6_loss: 0.0628 - dense_8_loss: 0.1386 - dense_2_acc: 0.9922 - dense_4_acc: 0.9531 - dense_6_acc: 0.9766 - dense_8_acc: 0.968 - ETA: 2s - loss: 0.1904 - dense_2_loss: 0.0204 - dense_4_loss: 0.0680 - dense_6_loss: 0.0456 - dense_8_loss: 0.0792 - dense_2_acc: 0.9961 - dense_4_acc: 0.9727 - dense_6_acc: 0.9883 - dense_8_acc: 0.984 - ETA: 2s - loss: 0.1786 - dense_2_loss: 0.0445 - dense_4_loss: 0.0681 - dense_6_loss: 0.0345 - dense_8_loss: 0.0570 - dense_2_acc: 0.9844 - dense_4_acc: 0.9740 - dense_6_acc: 0.9922 - dense_8_acc: 0.987 - ETA: 2s - loss: 0.1697 - dense_2_loss: 0.0423 - dense_4_loss: 0.0694 - dense_6_loss: 0.0322 - dense_8_loss: 0.0503 - dense_2_acc: 0.9824 - dense_4_acc: 0.9727 - dense_6_acc: 0.9902 - dense_8_acc: 0.988 - ETA: 2s - loss: 0.1582 - dense_2_loss: 0.0348 - dense_4_loss: 0.0621 - dense_6_loss: 0.0317 - dense_8_loss: 0.0516 - dense_2_acc: 0.9859 - 

4500/4500 [==============================] - ETA: 2s - loss: 0.1993 - dense_2_loss: 0.0483 - dense_4_loss: 0.0637 - dense_6_loss: 0.0707 - dense_8_loss: 0.0451 - dense_2_acc: 0.9766 - dense_4_acc: 0.9766 - dense_6_acc: 0.9766 - dense_8_acc: 0.984 - ETA: 2s - loss: 0.1477 - dense_2_loss: 0.0326 - dense_4_loss: 0.0614 - dense_6_loss: 0.0487 - dense_8_loss: 0.0269 - dense_2_acc: 0.9805 - dense_4_acc: 0.9805 - dense_6_acc: 0.9805 - dense_8_acc: 0.992 - ETA: 2s - loss: 0.1493 - dense_2_loss: 0.0395 - dense_4_loss: 0.0602 - dense_6_loss: 0.0450 - dense_8_loss: 0.0273 - dense_2_acc: 0.9818 - dense_4_acc: 0.9792 - dense_6_acc: 0.9844 - dense_8_acc: 0.989 - ETA: 2s - loss: 0.1541 - dense_2_loss: 0.0365 - dense_4_loss: 0.0599 - dense_6_loss: 0.0442 - dense_8_loss: 0.0359 - dense_2_acc: 0.9844 - dense_4_acc: 0.9785 - dense_6_acc: 0.9844 - dense_8_acc: 0.986 - ETA: 2s - loss: 0.1482 - dense_2_loss: 0.0349 - dense_4_loss: 0.0595 - dense_6_loss: 0.0379 - dense_8_loss: 0.0374 - dense_2_acc: 0.9859 - 

4500/4500 [==============================] - ETA: 2s - loss: 0.1171 - dense_2_loss: 0.0045 - dense_4_loss: 0.0742 - dense_6_loss: 0.0334 - dense_8_loss: 0.0214 - dense_2_acc: 1.0000 - dense_4_acc: 0.9688 - dense_6_acc: 0.9922 - dense_8_acc: 0.992 - ETA: 2s - loss: 0.1390 - dense_2_loss: 0.0307 - dense_4_loss: 0.0582 - dense_6_loss: 0.0382 - dense_8_loss: 0.0322 - dense_2_acc: 0.9844 - dense_4_acc: 0.9805 - dense_6_acc: 0.9922 - dense_8_acc: 0.988 - ETA: 2s - loss: 0.1589 - dense_2_loss: 0.0317 - dense_4_loss: 0.0927 - dense_6_loss: 0.0316 - dense_8_loss: 0.0276 - dense_2_acc: 0.9818 - dense_4_acc: 0.9714 - dense_6_acc: 0.9948 - dense_8_acc: 0.989 - ETA: 2s - loss: 0.1686 - dense_2_loss: 0.0296 - dense_4_loss: 0.0836 - dense_6_loss: 0.0404 - dense_8_loss: 0.0394 - dense_2_acc: 0.9844 - dense_4_acc: 0.9766 - dense_6_acc: 0.9883 - dense_8_acc: 0.988 - ETA: 2s - loss: 0.1669 - dense_2_loss: 0.0270 - dense_4_loss: 0.0927 - dense_6_loss: 0.0351 - dense_8_loss: 0.0367 - dense_2_acc: 0.9859 - 

In [ ]:
model.fit(train_data, [train_labels1, train_labels2, train_labels3, train_labels4],
          epochs=100,
          batch_size=128,
          validation_data=(dev_data, [dev_labels1, dev_labels2, dev_labels3, dev_labels4]),
          callbacks=[checkpoint,plateau], verbose=1, shuffle=True)

In [39]:
def decode(y):
    y = np.argmax(np.array(y), axis=-1)
    return ''.join([id2label[x] for x in y])

# 测试验证集
cy_list = []
py_list = []

print('TESTING...')
for i in range(len(dev_data)):
    X = np.asarray([dev_data[i]])
    y = [dev_labels1[i], dev_labels2[i], dev_labels3[i], dev_labels4[i]]
    py = np.squeeze(model.predict(X))
    cy_list.append(decode(y))
    py_list.append(decode(py))

current_num = [1 if cy == py else 0 for cy, py in zip(cy_list, py_list)]
print('FINAL MODEL ACC: {}'.format(sum(current_num)/len(cy_list)))

TESTING...
FINAL MODEL ACC: 0.94


## 模型1

In [20]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np
import h5py
import os
import keras 
from keras.utils import np_utils, conv_utils
from keras.models import Model
from keras.layers import Flatten, Dense, Input 
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50
from keras import backend as K
from keras.callbacks import *

In [23]:
pic_in = Input( shape=(height, width, 3))
# Block 1
cnn_features = Conv2D(32, (3,3), activation='relu', padding='same')(pic_in)
cnn_features = Conv2D(32, (3,3), activation='relu')(cnn_features)
cnn_features = MaxPooling2D(2)(cnn_features)
cnn_features = Dropout(0.25)(cnn_features)
# Block 2
cnn_features = Conv2D(64, (3,3), activation='relu',padding='same')(cnn_features)
cnn_features = Conv2D(64, (3,3), activation='relu')(cnn_features)
cnn_features = MaxPooling2D(2)(cnn_features)
cnn_features = Dropout(0.25)(cnn_features)
# Block 3
cnn_features = Conv2D(128, (3,3), activation='relu',padding='same')(cnn_features)
cnn_features = Conv2D(128, (3,3), activation='relu')(cnn_features)
cnn_features = MaxPooling2D(2)(cnn_features)
cnn_features = Dropout(0.25)(cnn_features)
cnn_features = Flatten()(cnn_features)

# classifier 1
output_l1 = Dense(128, activation='relu')(cnn_features)
output_l1 = Dropout(0.5)(output_l1)
output_l1 = Dense(word_class, activation='sigmoid')(output_l1)
# classifier 2
output_l2 = Dense(128, activation='relu')(cnn_features)
output_l2 = Dropout(0.5)(output_l2)
output_l2 = Dense(word_class, activation='sigmoid')(output_l2)
# classifier 3
output_l3 = Dense(128, activation='relu')(cnn_features)
output_l3 = Dropout(0.5)(output_l3)
output_l3 = Dense(word_class, activation='sigmoid')(output_l3)
# classifier 4
output_l4 = Dense(128, activation='relu')(cnn_features)
output_l4 = Dropout(0.5)(output_l4)
output_l4 = Dense(word_class, activation='sigmoid')(output_l4)

model = Model(inputs=pic_in, outputs=[output_l1, output_l2, output_l3, output_l4])

In [24]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 40, 120, 3)   0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 40, 120, 32)  896         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 38, 118, 32)  9248        conv2d_21[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_11 (MaxPooling2D) (None, 19, 59, 32)   0           conv2d_22[0][0]                  
__________________________________________________________________________________________________
dropout_18

In [18]:
adam = Adam(lr=0.00001)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              loss_weights=[1., 1., 1., 1.], # 给四个分类器同样的权重
              metrics=['accuracy'])

print('TRAINING...')
#plateau = ReduceLROnPlateau(monitor="val_loss", verbose=1, mode='min', factor=0.5, patience=2)
checkpoint = ModelCheckpoint('model.hdf5', monitor='val_loss', 
                                 verbose=2, save_best_only=True, mode='min')
#keras.optimizers.SGD(lr=0.001, momentum=0.01, decay=0.001, nesterov=True)
model.fit(train_data, [train_labels1, train_labels2, train_labels3, train_labels4],
          epochs=50,
          batch_size=128,
          validation_data=(dev_data, [dev_labels1, dev_labels2, dev_labels3, dev_labels4]),
          callbacks=[checkpoint], verbose=1, shuffle=True)

NameError: name 'Adam' is not defined

In [25]:
def decode(y):
    y = np.argmax(np.array(y), axis=-1)
    return ''.join([id2label[x] for x in y])

# 测试验证集
cy_list = []
py_list = []

print('TESTING...')
for i in range(len(dev_data)):
    X = np.asarray([dev_data[i]])
    y = [dev_labels1[i], dev_labels2[i], dev_labels3[i], dev_labels4[i]]
    py = np.squeeze(model.predict(X))
    cy_list.append(decode(y))
    py_list.append(decode(py))

current_num = [1 if cy == py else 0 for cy, py in zip(cy_list, py_list)]
print('FINAL MODEL ACC: {}'.format(sum(current_num)/len(cy_list)))

TESTING...
FINAL MODEL ACC: 0.916


In [26]:
py_list[:5]

['4Jy3', 'nbKh', 'J3da', '02hu', '3SjC']

In [27]:
cy_list[:5]

['4Jy3', 'nbKh', 'J3da', '02hu', '3SjC']

In [7]:
import numpy as np
import os
import logging
from keras.models import load_model  # 一系列网络层按顺序构成的栈
import glob
from keras.preprocessing import image
import tensorflow as tf

In [13]:
import tensorflow

In [15]:
keras.__version__

'2.2.4'

In [1]:
import numpy as np
import os
import logging
from keras.models import load_model  # 一系列网络层按顺序构成的栈
import glob
from keras.preprocessing import image
import tensorflow as tf

def decode(y):
    
    y = np.argmax(np.array(y), axis=-1)
    return ''.join([id2label[x] for x in y])
test_image_list =  glob.glob(r'test/*.jpg')

Using TensorFlow backend.


In [3]:
len(test_image_list)

5000

In [85]:
import numpy as np
import os
import logging
from keras.models import load_model  # 一系列网络层按顺序构成的栈
import glob
from keras.preprocessing import image
import tensorflow as tf

def decode(y):
    
    y = np.argmax(np.array(y), axis=-1)
    return ''.join([id2label[x] for x in y])
test_image_list =  glob.glob(r'test/*.jpg')

logger = logging.getLogger("forecast by model")
# 每个验证码所包含的字符数
#日志初始化
def init_logger():
    logging.basicConfig(
        format='%(asctime)s : %(levelname)s : %(message)s',
        level=logging.DEBUG,
        handlers=[logging.StreamHandler()])  
# 图片总数
image_count = 0
test_data = pd.DataFrame(columns=['ID','label'])
# 图片
for i, img in enumerate(image_list):
    img_path = test_image_list[i]
    image_path = test_image_list[i].split('\\')[1]
    # 读取图片
    raw_img = image.load_img(img_path, target_size=(height, width))
    X_test = np.zeros((1, height, width, 3), dtype=np.float32)
    X_test[0] = image.img_to_array(raw_img)
    # 预测
    py = np.squeeze(model.predict(X_test))
    result = decode(py)
    logger.debug("预测的结果为" + result)
    P = pd.DataFrame([[image_path,result]],columns=['ID','label'])
    test_data = test_data.append(P)

In [45]:
test_data.head(n=5)

,ID,label
0,1.jpg,s1O2
0,10.jpg,8bud
0,100.jpg,g16l
0,1000.jpg,hFSU
0,1001.jpg,whwJ


In [33]:
test_data.head(n=5)

,ID,label
0,1.jpg,s1O2
0,10.jpg,8bud
0,100.jpg,g16l
0,1000.jpg,hFSU
0,1001.jpg,whwJ


In [86]:
test_data.to_csv('20191122_cnn_942.csv',index=None)

## 模型二

In [15]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np
import h5py
import os
import keras 
from keras.utils import np_utils, conv_utils
from keras.models import Model
from keras.layers import Flatten, Dense, Input 
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50
from keras import backend as K
from keras.callbacks import *

In [24]:
from keras import regularizers
pic_in = Input( shape=(height, width, 3))
# Block 1
cnn_features = Conv2D(32, (3,3), activation='relu', padding='same',kernel_regularizer = regularizers.L1L2(l2=0.0001))(pic_in)
cnn_features = Conv2D(32, (3,3), activation='relu',kernel_regularizer = regularizers.L1L2(l2=0.0001))(cnn_features)
cnn_features = MaxPooling2D((2, 2))(cnn_features)
cnn_features = Dropout(0.25)(cnn_features)
# Block 2
cnn_features = Conv2D(64, (3,3), activation='relu',padding='same',kernel_regularizer = regularizers.L1L2(l2=0.0001))(cnn_features)
cnn_features = Conv2D(64, (3,3), activation='relu',kernel_regularizer = regularizers.L1L2(l2=0.0001))(cnn_features)
cnn_features = MaxPooling2D((2, 2))(cnn_features)
cnn_features = Dropout(0.25)(cnn_features)
# Block 3
cnn_features = Conv2D(128, (3,3), activation='relu',padding='same',kernel_regularizer = regularizers.L1L2(l2=0.0001))(cnn_features)
cnn_features = Conv2D(128, (3,3), activation='relu',kernel_regularizer = regularizers.L1L2(l2=0.0001))(cnn_features)
cnn_features = MaxPooling2D((2, 2))(cnn_features)
cnn_features = Dropout(0.25)(cnn_features)
cnn_features = Flatten()(cnn_features)
# classifier 1
output_l1 = Dense(128, activation='relu',kernel_regularizer = regularizers.L1L2(l2=0.0001))(cnn_features)
output_l1 = Dropout(0.5)(output_l1)
output_l1 = Dense(word_class, activation='sigmoid')(output_l1)
# classifier 2
output_l2 = Dense(128, activation='relu',kernel_regularizer = regularizers.L1L2(l2=0.0001))(cnn_features)
output_l2 = Dropout(0.5)(output_l2)
output_l2 = Dense(word_class, activation='sigmoid')(output_l2)
# classifier 3
output_l3 = Dense(128, activation='relu',kernel_regularizer = regularizers.L1L2(l2=0.0001))(cnn_features)
output_l3 = Dropout(0.5)(output_l3)
output_l3 = Dense(word_class, activation='sigmoid')(output_l3)
# classifier 4
output_l4 = Dense(128, activation='relu',kernel_regularizer = regularizers.L1L2(l2=0.0001))(cnn_features)
output_l4 = Dropout(0.5)(output_l4)
output_l4 = Dense(word_class, activation='sigmoid')(output_l4)

model = Model(inputs=pic_in, outputs=[output_l1, output_l2, output_l3, output_l4])

In [28]:
adam = Adam(lr=0.00000001)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              loss_weights=[1., 1., 1., 1.], # 给四个分类器同样的权重
              metrics=['accuracy'])

print('TRAINING...')
plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=20, verbose=1, mode='auto')
#plateau = ReduceLROnPlateau(monitor="val_loss", verbose=1, mode='min', factor=0.5, patience=2)
checkpoint = ModelCheckpoint('model.hdf5', monitor='val_loss', 
                                 verbose=2, save_best_only=True, mode='min')
keras.optimizers.SGD(lr=0.0000001, momentum=0.000001, decay=0.00001, nesterov=True)
model.fit(train_data, [train_labels1, train_labels2, train_labels3, train_labels4],
          epochs=100,
          batch_size=128,
          validation_data=(dev_data, [dev_labels1, dev_labels2, dev_labels3, dev_labels4]),
          callbacks=[checkpoint,plateau], verbose=1, shuffle=True)

TRAINING...
Train on 4500 samples, validate on 500 samples
Epoch 1/100
4500/4500 [==============================] - ETA: 1:03 - loss: 0.6128 - dense_20_loss: 0.0728 - dense_22_loss: 0.1211 - dense_24_loss: 0.0467 - dense_26_loss: 0.0461 - dense_20_acc: 0.9766 - dense_22_acc: 0.9609 - dense_24_acc: 0.9922 - dense_26_acc: 0.976 - ETA: 31s - loss: 0.5605 - dense_20_loss: 0.0561 - dense_22_loss: 0.0892 - dense_24_loss: 0.0527 - dense_26_loss: 0.0363 - dense_20_acc: 0.9805 - dense_22_acc: 0.9688 - dense_24_acc: 0.9844 - dense_26_acc: 0.980 - ETA: 21s - loss: 0.5469 - dense_20_loss: 0.0523 - dense_22_loss: 0.0845 - dense_24_loss: 0.0507 - dense_26_loss: 0.0333 - dense_20_acc: 0.9792 - dense_22_acc: 0.9688 - dense_24_acc: 0.9844 - dense_26_acc: 0.98 - ETA: 16s - loss: 0.5475 - dense_20_loss: 0.0417 - dense_22_loss: 0.0800 - dense_24_loss: 0.0715 - dense_26_loss: 0.0281 - dense_20_acc: 0.9844 - dense_22_acc: 0.9668 - dense_24_acc: 0.9746 - dense_26_acc: 0.98 - ETA: 13s - loss: 0.5680 - dense_2

4500/4500 [==============================] - ETA: 2s - loss: 0.5850 - dense_20_loss: 0.0418 - dense_22_loss: 0.0683 - dense_24_loss: 0.0703 - dense_26_loss: 0.0786 - dense_20_acc: 0.9844 - dense_22_acc: 0.9766 - dense_24_acc: 0.9766 - dense_26_acc: 0.976 - ETA: 2s - loss: 0.5634 - dense_20_loss: 0.0564 - dense_22_loss: 0.0704 - dense_24_loss: 0.0519 - dense_26_loss: 0.0585 - dense_20_acc: 0.9805 - dense_22_acc: 0.9766 - dense_24_acc: 0.9844 - dense_26_acc: 0.980 - ETA: 2s - loss: 0.5729 - dense_20_loss: 0.0643 - dense_22_loss: 0.0622 - dense_24_loss: 0.0721 - dense_26_loss: 0.0481 - dense_20_acc: 0.9818 - dense_22_acc: 0.9844 - dense_24_acc: 0.9766 - dense_26_acc: 0.984 - ETA: 2s - loss: 0.5744 - dense_20_loss: 0.0624 - dense_22_loss: 0.0702 - dense_24_loss: 0.0600 - dense_26_loss: 0.0557 - dense_20_acc: 0.9824 - dense_22_acc: 0.9805 - dense_24_acc: 0.9824 - dense_26_acc: 0.982 - ETA: 2s - loss: 0.5756 - dense_20_loss: 0.0610 - dense_22_loss: 0.0669 - dense_24_loss: 0.0668 - dense_26_l

4500/4500 [==============================] - ETA: 2s - loss: 0.5624 - dense_20_loss: 0.0181 - dense_22_loss: 0.0924 - dense_24_loss: 0.1154 - dense_26_loss: 0.0104 - dense_20_acc: 1.0000 - dense_22_acc: 0.9531 - dense_24_acc: 0.9453 - dense_26_acc: 1.000 - ETA: 2s - loss: 0.6512 - dense_20_loss: 0.0298 - dense_22_loss: 0.1087 - dense_24_loss: 0.1265 - dense_26_loss: 0.0600 - dense_20_acc: 0.9922 - dense_22_acc: 0.9570 - dense_24_acc: 0.9570 - dense_26_acc: 0.976 - ETA: 2s - loss: 0.6326 - dense_20_loss: 0.0328 - dense_22_loss: 0.1033 - dense_24_loss: 0.1032 - dense_26_loss: 0.0672 - dense_20_acc: 0.9922 - dense_22_acc: 0.9583 - dense_24_acc: 0.9609 - dense_26_acc: 0.974 - ETA: 2s - loss: 0.6292 - dense_20_loss: 0.0319 - dense_22_loss: 0.1084 - dense_24_loss: 0.1057 - dense_26_loss: 0.0570 - dense_20_acc: 0.9922 - dense_22_acc: 0.9609 - dense_24_acc: 0.9629 - dense_26_acc: 0.978 - ETA: 2s - loss: 0.6140 - dense_20_loss: 0.0303 - dense_22_loss: 0.1034 - dense_24_loss: 0.1053 - dense_26_l

4500/4500 [==============================] - ETA: 2s - loss: 0.5215 - dense_20_loss: 0.0743 - dense_22_loss: 0.0432 - dense_24_loss: 0.0394 - dense_26_loss: 0.0385 - dense_20_acc: 0.9688 - dense_22_acc: 0.9844 - dense_24_acc: 0.9844 - dense_26_acc: 0.976 - ETA: 2s - loss: 0.5666 - dense_20_loss: 0.0496 - dense_22_loss: 0.0657 - dense_24_loss: 0.0283 - dense_26_loss: 0.0968 - dense_20_acc: 0.9844 - dense_22_acc: 0.9727 - dense_24_acc: 0.9922 - dense_26_acc: 0.964 - ETA: 2s - loss: 0.5911 - dense_20_loss: 0.0611 - dense_22_loss: 0.0705 - dense_24_loss: 0.0547 - dense_26_loss: 0.0786 - dense_20_acc: 0.9818 - dense_22_acc: 0.9766 - dense_24_acc: 0.9818 - dense_26_acc: 0.971 - ETA: 2s - loss: 0.5577 - dense_20_loss: 0.0488 - dense_22_loss: 0.0606 - dense_24_loss: 0.0602 - dense_26_loss: 0.0620 - dense_20_acc: 0.9844 - dense_22_acc: 0.9824 - dense_24_acc: 0.9805 - dense_26_acc: 0.976 - ETA: 2s - loss: 0.5631 - dense_20_loss: 0.0416 - dense_22_loss: 0.0711 - dense_24_loss: 0.0672 - dense_26_l

4500/4500 [==============================] - ETA: 2s - loss: 0.6887 - dense_20_loss: 0.0603 - dense_22_loss: 0.0783 - dense_24_loss: 0.1099 - dense_26_loss: 0.1140 - dense_20_acc: 0.9688 - dense_22_acc: 0.9609 - dense_24_acc: 0.9766 - dense_26_acc: 0.960 - ETA: 2s - loss: 0.5948 - dense_20_loss: 0.0511 - dense_22_loss: 0.0616 - dense_24_loss: 0.0838 - dense_26_loss: 0.0721 - dense_20_acc: 0.9805 - dense_22_acc: 0.9766 - dense_24_acc: 0.9766 - dense_26_acc: 0.980 - ETA: 2s - loss: 0.5969 - dense_20_loss: 0.0656 - dense_22_loss: 0.0703 - dense_24_loss: 0.0800 - dense_26_loss: 0.0549 - dense_20_acc: 0.9818 - dense_22_acc: 0.9792 - dense_24_acc: 0.9792 - dense_26_acc: 0.984 - ETA: 2s - loss: 0.6150 - dense_20_loss: 0.0614 - dense_22_loss: 0.0887 - dense_24_loss: 0.0883 - dense_26_loss: 0.0504 - dense_20_acc: 0.9805 - dense_22_acc: 0.9707 - dense_24_acc: 0.9766 - dense_26_acc: 0.982 - ETA: 2s - loss: 0.6098 - dense_20_loss: 0.0527 - dense_22_loss: 0.1003 - dense_24_loss: 0.0856 - dense_26_l

KeyboardInterrupt: 

In [26]:
model.fit(train_data, [train_labels1, train_labels2, train_labels3, train_labels4],
          epochs=1000,
          batch_size=128,
          validation_data=(dev_data, [dev_labels1, dev_labels2, dev_labels3, dev_labels4]),
          callbacks=[checkpoint,plateau], verbose=1, shuffle=True)

Train on 4500 samples, validate on 500 samples
Epoch 1/1000
4500/4500 [==============================] - ETA: 3s - loss: 0.6200 - dense_20_loss: 0.0805 - dense_22_loss: 0.0992 - dense_24_loss: 0.0254 - dense_26_loss: 0.0645 - dense_20_acc: 0.9609 - dense_22_acc: 0.9688 - dense_24_acc: 0.9922 - dense_26_acc: 0.992 - ETA: 2s - loss: 0.6265 - dense_20_loss: 0.0696 - dense_22_loss: 0.0958 - dense_24_loss: 0.0442 - dense_26_loss: 0.0665 - dense_20_acc: 0.9688 - dense_22_acc: 0.9688 - dense_24_acc: 0.9844 - dense_26_acc: 0.984 - ETA: 2s - loss: 0.6074 - dense_20_loss: 0.0667 - dense_22_loss: 0.0959 - dense_24_loss: 0.0460 - dense_26_loss: 0.0483 - dense_20_acc: 0.9714 - dense_22_acc: 0.9688 - dense_24_acc: 0.9844 - dense_26_acc: 0.989 - ETA: 2s - loss: 0.6167 - dense_20_loss: 0.0645 - dense_22_loss: 0.0896 - dense_24_loss: 0.0616 - dense_26_loss: 0.0505 - dense_20_acc: 0.9746 - dense_22_acc: 0.9746 - dense_24_acc: 0.9805 - dense_26_acc: 0.988 - ETA: 2s - loss: 0.6486 - dense_20_loss: 0.0614 

4500/4500 [==============================] - ETA: 2s - loss: 0.5833 - dense_20_loss: 0.0718 - dense_22_loss: 0.0975 - dense_24_loss: 0.0420 - dense_26_loss: 0.0218 - dense_20_acc: 0.9844 - dense_22_acc: 0.9766 - dense_24_acc: 0.9844 - dense_26_acc: 1.000 - ETA: 2s - loss: 0.6317 - dense_20_loss: 0.0596 - dense_22_loss: 0.1137 - dense_24_loss: 0.0375 - dense_26_loss: 0.0707 - dense_20_acc: 0.9844 - dense_22_acc: 0.9492 - dense_24_acc: 0.9844 - dense_26_acc: 0.992 - ETA: 2s - loss: 0.6660 - dense_20_loss: 0.0574 - dense_22_loss: 0.1154 - dense_24_loss: 0.0562 - dense_26_loss: 0.0867 - dense_20_acc: 0.9818 - dense_22_acc: 0.9557 - dense_24_acc: 0.9792 - dense_26_acc: 0.987 - ETA: 2s - loss: 0.6647 - dense_20_loss: 0.0664 - dense_22_loss: 0.1003 - dense_24_loss: 0.0762 - dense_26_loss: 0.0715 - dense_20_acc: 0.9785 - dense_22_acc: 0.9609 - dense_24_acc: 0.9746 - dense_26_acc: 0.990 - ETA: 2s - loss: 0.6732 - dense_20_loss: 0.0693 - dense_22_loss: 0.0968 - dense_24_loss: 0.0792 - dense_26_l

4500/4500 [==============================] - ETA: 2s - loss: 0.6220 - dense_20_loss: 0.0733 - dense_22_loss: 0.0815 - dense_24_loss: 0.0791 - dense_26_loss: 0.0378 - dense_20_acc: 0.9609 - dense_22_acc: 0.9844 - dense_24_acc: 0.9844 - dense_26_acc: 0.992 - ETA: 2s - loss: 0.6694 - dense_20_loss: 0.0576 - dense_22_loss: 0.1114 - dense_24_loss: 0.0896 - dense_26_loss: 0.0607 - dense_20_acc: 0.9688 - dense_22_acc: 0.9688 - dense_24_acc: 0.9688 - dense_26_acc: 0.980 - ETA: 2s - loss: 0.6865 - dense_20_loss: 0.0695 - dense_22_loss: 0.1174 - dense_24_loss: 0.0777 - dense_26_loss: 0.0719 - dense_20_acc: 0.9688 - dense_22_acc: 0.9661 - dense_24_acc: 0.9714 - dense_26_acc: 0.976 - ETA: 2s - loss: 0.6820 - dense_20_loss: 0.0607 - dense_22_loss: 0.1159 - dense_24_loss: 0.0960 - dense_26_loss: 0.0593 - dense_20_acc: 0.9766 - dense_22_acc: 0.9629 - dense_24_acc: 0.9707 - dense_26_acc: 0.982 - ETA: 2s - loss: 0.6885 - dense_20_loss: 0.0641 - dense_22_loss: 0.1155 - dense_24_loss: 0.1003 - dense_26_l

4500/4500 [==============================] - ETA: 2s - loss: 0.6689 - dense_20_loss: 0.0412 - dense_22_loss: 0.0880 - dense_24_loss: 0.1386 - dense_26_loss: 0.0513 - dense_20_acc: 0.9844 - dense_22_acc: 0.9844 - dense_24_acc: 0.9453 - dense_26_acc: 0.984 - ETA: 2s - loss: 0.6288 - dense_20_loss: 0.0457 - dense_22_loss: 0.0753 - dense_24_loss: 0.1170 - dense_26_loss: 0.0409 - dense_20_acc: 0.9883 - dense_22_acc: 0.9844 - dense_24_acc: 0.9609 - dense_26_acc: 0.988 - ETA: 2s - loss: 0.6063 - dense_20_loss: 0.0481 - dense_22_loss: 0.0697 - dense_24_loss: 0.0920 - dense_26_loss: 0.0467 - dense_20_acc: 0.9844 - dense_22_acc: 0.9818 - dense_24_acc: 0.9688 - dense_26_acc: 0.987 - ETA: 2s - loss: 0.6572 - dense_20_loss: 0.0556 - dense_22_loss: 0.0887 - dense_24_loss: 0.0998 - dense_26_loss: 0.0632 - dense_20_acc: 0.9824 - dense_22_acc: 0.9727 - dense_24_acc: 0.9648 - dense_26_acc: 0.982 - ETA: 2s - loss: 0.6450 - dense_20_loss: 0.0594 - dense_22_loss: 0.0820 - dense_24_loss: 0.0986 - dense_26_l

4500/4500 [==============================] - ETA: 2s - loss: 0.7848 - dense_20_loss: 0.0171 - dense_22_loss: 0.1957 - dense_24_loss: 0.1542 - dense_26_loss: 0.0683 - dense_20_acc: 1.0000 - dense_22_acc: 0.9688 - dense_24_acc: 0.9609 - dense_26_acc: 0.968 - ETA: 2s - loss: 0.7261 - dense_20_loss: 0.0402 - dense_22_loss: 0.1780 - dense_24_loss: 0.1161 - dense_26_loss: 0.0421 - dense_20_acc: 0.9883 - dense_22_acc: 0.9531 - dense_24_acc: 0.9727 - dense_26_acc: 0.984 - ETA: 2s - loss: 0.6699 - dense_20_loss: 0.0336 - dense_22_loss: 0.1429 - dense_24_loss: 0.1047 - dense_26_loss: 0.0392 - dense_20_acc: 0.9896 - dense_22_acc: 0.9609 - dense_24_acc: 0.9688 - dense_26_acc: 0.987 - ETA: 2s - loss: 0.6539 - dense_20_loss: 0.0422 - dense_22_loss: 0.1279 - dense_24_loss: 0.0937 - dense_26_loss: 0.0405 - dense_20_acc: 0.9844 - dense_22_acc: 0.9609 - dense_24_acc: 0.9707 - dense_26_acc: 0.986 - ETA: 2s - loss: 0.6773 - dense_20_loss: 0.0399 - dense_22_loss: 0.1435 - dense_24_loss: 0.0977 - dense_26_l

...

4500/4500 [==============================] - ETA: 2s - loss: 0.5598 - dense_20_loss: 0.0558 - dense_22_loss: 0.0868 - dense_24_loss: 0.0348 - dense_26_loss: 0.0350 - dense_20_acc: 0.9844 - dense_22_acc: 0.9609 - dense_24_acc: 0.9766 - dense_26_acc: 0.984 - ETA: 2s - loss: 0.5889 - dense_20_loss: 0.0750 - dense_22_loss: 0.0589 - dense_24_loss: 0.0395 - dense_26_loss: 0.0682 - dense_20_acc: 0.9688 - dense_22_acc: 0.9727 - dense_24_acc: 0.9805 - dense_26_acc: 0.980 - ETA: 2s - loss: 0.5841 - dense_20_loss: 0.0708 - dense_22_loss: 0.0615 - dense_24_loss: 0.0458 - dense_26_loss: 0.0587 - dense_20_acc: 0.9740 - dense_22_acc: 0.9766 - dense_24_acc: 0.9766 - dense_26_acc: 0.984 - ETA: 2s - loss: 0.5882 - dense_20_loss: 0.0592 - dense_22_loss: 0.0657 - dense_24_loss: 0.0555 - dense_26_loss: 0.0604 - dense_20_acc: 0.9785 - dense_22_acc: 0.9785 - dense_24_acc: 0.9727 - dense_26_acc: 0.984 - ETA: 2s - loss: 0.5848 - dense_20_loss: 0.0531 - dense_22_loss: 0.0694 - dense_24_loss: 0.0587 - dense_26_l

4500/4500 [==============================] - ETA: 2s - loss: 0.5943 - dense_20_loss: 0.0309 - dense_22_loss: 0.1125 - dense_24_loss: 0.0534 - dense_26_loss: 0.0502 - dense_20_acc: 0.9922 - dense_22_acc: 0.9531 - dense_24_acc: 0.9766 - dense_26_acc: 0.976 - ETA: 2s - loss: 0.6191 - dense_20_loss: 0.0306 - dense_22_loss: 0.0986 - dense_24_loss: 0.0535 - dense_26_loss: 0.0890 - dense_20_acc: 0.9883 - dense_22_acc: 0.9609 - dense_24_acc: 0.9844 - dense_26_acc: 0.984 - ETA: 2s - loss: 0.6052 - dense_20_loss: 0.0290 - dense_22_loss: 0.0957 - dense_24_loss: 0.0638 - dense_26_loss: 0.0693 - dense_20_acc: 0.9870 - dense_22_acc: 0.9661 - dense_24_acc: 0.9844 - dense_26_acc: 0.987 - ETA: 2s - loss: 0.6103 - dense_20_loss: 0.0400 - dense_22_loss: 0.0902 - dense_24_loss: 0.0713 - dense_26_loss: 0.0615 - dense_20_acc: 0.9844 - dense_22_acc: 0.9688 - dense_24_acc: 0.9766 - dense_26_acc: 0.986 - ETA: 2s - loss: 0.6059 - dense_20_loss: 0.0396 - dense_22_loss: 0.0880 - dense_24_loss: 0.0742 - dense_26_l

4500/4500 [==============================] - ETA: 2s - loss: 0.6559 - dense_20_loss: 0.0120 - dense_22_loss: 0.2027 - dense_24_loss: 0.0660 - dense_26_loss: 0.0280 - dense_20_acc: 1.0000 - dense_22_acc: 0.9297 - dense_24_acc: 0.9844 - dense_26_acc: 0.992 - ETA: 2s - loss: 0.5957 - dense_20_loss: 0.0182 - dense_22_loss: 0.1329 - dense_24_loss: 0.0605 - dense_26_loss: 0.0368 - dense_20_acc: 1.0000 - dense_22_acc: 0.9609 - dense_24_acc: 0.9844 - dense_26_acc: 0.984 - ETA: 2s - loss: 0.6189 - dense_20_loss: 0.0357 - dense_22_loss: 0.1082 - dense_24_loss: 0.0707 - dense_26_loss: 0.0570 - dense_20_acc: 0.9870 - dense_22_acc: 0.9661 - dense_24_acc: 0.9766 - dense_26_acc: 0.981 - ETA: 2s - loss: 0.6110 - dense_20_loss: 0.0503 - dense_22_loss: 0.0932 - dense_24_loss: 0.0706 - dense_26_loss: 0.0496 - dense_20_acc: 0.9824 - dense_22_acc: 0.9707 - dense_24_acc: 0.9766 - dense_26_acc: 0.984 - ETA: 2s - loss: 0.6131 - dense_20_loss: 0.0533 - dense_22_loss: 0.0870 - dense_24_loss: 0.0757 - dense_26_l

4500/4500 [==============================] - ETA: 2s - loss: 0.7016 - dense_20_loss: 0.0973 - dense_22_loss: 0.0563 - dense_24_loss: 0.1086 - dense_26_loss: 0.0921 - dense_20_acc: 0.9844 - dense_22_acc: 0.9922 - dense_24_acc: 0.9766 - dense_26_acc: 0.960 - ETA: 2s - loss: 0.6492 - dense_20_loss: 0.0634 - dense_22_loss: 0.0629 - dense_24_loss: 0.1033 - dense_26_loss: 0.0723 - dense_20_acc: 0.9883 - dense_22_acc: 0.9883 - dense_24_acc: 0.9727 - dense_26_acc: 0.968 - ETA: 2s - loss: 0.6056 - dense_20_loss: 0.0459 - dense_22_loss: 0.0667 - dense_24_loss: 0.0806 - dense_26_loss: 0.0651 - dense_20_acc: 0.9922 - dense_22_acc: 0.9844 - dense_24_acc: 0.9818 - dense_26_acc: 0.976 - ETA: 2s - loss: 0.6243 - dense_20_loss: 0.0455 - dense_22_loss: 0.0951 - dense_24_loss: 0.0707 - dense_26_loss: 0.0657 - dense_20_acc: 0.9922 - dense_22_acc: 0.9785 - dense_24_acc: 0.9805 - dense_26_acc: 0.976 - ETA: 2s - loss: 0.6173 - dense_20_loss: 0.0455 - dense_22_loss: 0.0921 - dense_24_loss: 0.0710 - dense_26_l

KeyboardInterrupt: 

# 模型五

In [ ]:
pic_in = Input( shape=(height, width, 3))
# Block 1
cnn_features = Conv2D(32, (3,3), activation='relu', padding='same')(pic_in)
cnn_features = Conv2D(32, (3,3), activation='relu')(cnn_features)
cnn_features = MaxPooling2D((2, 2))(cnn_features)
cnn_features = Dropout(0.25)(cnn_features)
# Block 2
cnn_features = Conv2D(64, (3,3), activation='relu',padding='same')(cnn_features)
cnn_features = Conv2D(64, (3,3), activation='relu')(cnn_features)
cnn_features = MaxPooling2D((2, 2))(cnn_features)
cnn_features = Dropout(0.25)(cnn_features)
# Block 3
cnn_features = Conv2D(128, (3,3), activation='relu',padding='same')(cnn_features)
cnn_features = Conv2D(128, (3,3), activation='relu')(cnn_features)
cnn_features = MaxPooling2D((2, 2))(cnn_features)
cnn_features = Dropout(0.25)(cnn_features)
x = Flatten()(x)
#为输入数据施加Dropout。Dropout将在训练过程中每次更新参数时随机断开一定百分比（rate）的输入神经元，Dropout层用于防止过拟合。
x = Dropout(0.25)(x)
x= BatchNormalization()(x)
#Dense就是常用的全连接层
#最后连接4个分类器，每个分类器是56个神经元，分别输出56个字符的概率。
x = [Dense(word_class, activation='softmax', name='c%d'%(i+1))(x) for i in range(word_len)]
# x = [Dense(word_class, activation='sigmoid', name='c%d'%(i+1))(x) for i in range(word_len)]
output = concatenate(x)
#构建模型
model = Model(inputs=input_tensor, outputs=output)
# model.add(BatchNormalization())
#因为训练可能需要数个小时，所以这里加载了之前我训练好的参数。准确率为94%
#可以直接使用此参数继续进行训练，也可以自己从头开始训练
# model.load_weights('model/weights.10--6.55-0.3062.hdf5')

#这里优化器选用Adadelta，学习率0.1
opt = Adadelta(lr=0.01)
#编译模型以供训练，损失函数使用 categorical_crossentropy，使用accuracy评估模型在训练和测试时的性能的指标
#这里使用自定义的评估模型
model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics=['accuracy',custom_accuracy])
#每次epoch都保存一下权重，用于继续训练
checkpointer = ModelCheckpoint(filepath="model/weights.{epoch:02d}--{val_loss:.2f}-{val_acc:.4f}.hdf5",
                               verbose=2, save_weights_only=True)
#开始训练，validation_split代表10%的数据不参与训练，用于做验证急
#我之前训练了50个epochs以上，这里根据自己的情况进行选择。如果输出的val_acc已经达到你满意的数值，可以终止训练
model.fit(X, y, epochs= 1000,callbacks=[checkpointer], validation_split=0.1)
# plot model
plot_model(model, to_file='model/model1.png', show_shapes=True)



In [ ]:
#保存权重和模型
model.save_weights('model/captcha_model_weights1.h5')
model.save('model/captcha__model_21.h5')

# 模型六

In [1]:
import tensorflow as tf
import cv2 

In [ ]:
class TFRecordWriter:
    def __init__(self,tfrecord_path,labels_list,batch_size=64):
        self.labels_list = labels_list
        self.tfrecord_path = tfrecord_path
        self.batch_size = batch_size
    def write(self):
        writer = tf.python_io.TFRecordWriter(self.tfrecord_path)
        for item in self.labels_list: # 对label遍历
            img_path = item[0]
            label = item[1]
            img = cv2.imread(img_path)
            img = cv2.
            
    

# 测试

In [42]:
import numpy as np
import os
import logging
from keras.models import load_model  # 一系列网络层按顺序构成的栈
import glob
from keras.preprocessing import image
import tensorflow as tf

In [43]:
logger = logging.getLogger("forecast by model")
# 每个验证码所包含的字符数

In [44]:
word_len = 4
image_path = 'test/'
# 验证码所包含的字符 _表示未知
captcha_word = ['0','1','2','3','4','5','6','7','8','9',
           'A','B','C','D','E','F','G','H','I','J',
           'K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
           'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p',
           'q','r','s','t','u','v','w','x','y','z'
           ]

In [45]:
# 字符总数
word_class = len(captcha_word)

#日志初始化
def init_logger():
    logging.basicConfig(
        format='%(asctime)s : %(levelname)s : %(message)s',
        level=logging.DEBUG,
        handlers=[logging.StreamHandler()])

def custom_accuracy(y_true, y_pred):
    predict = tf.reshape(y_pred, [-1, word_len, word_class])
    max_idx_p = tf.argmax(predict, 2)#这个做法牛逼，不用再做stack和reshape了，2，是在Charset那个维度上
    max_idx_l = tf.argmax(tf.reshape(y_true, [-1, word_len,word_class]), 2)
    correct_pred = tf.equal(max_idx_p, max_idx_l)
    _result = tf.map_fn(fn=lambda e: tf.reduce_all(e),elems=correct_pred,dtype=tf.bool)
    return tf.reduce_mean(tf.cast(_result, tf.float32))
# load json and create model
def create_model():
    weight_path = 'model/model2/captcha__model_2.h5'
    model = load_model(weight_path,
                        custom_objects={'custom_accuracy': custom_accuracy})
    return model

# 把数组转换回文字
def vec_to_captcha(vec):
    text = []
    # 把概率小于0.5的改为0，标记为错误
    vec[vec < 0.5] = 0

    char_pos = vec.nonzero()[0]

    for i, ch in enumerate(char_pos):
        text.append(captcha_word[ch % word_class])
    return ''.join(text)

In [46]:
init_logger()
#model = create_model()

In [47]:
test_image_list =  glob.glob(r'test/*.jpg')

In [48]:
# 图片总数
image_count = 0
test_data = pd.DataFrame(columns=['ID','label'])
# 图片
for i, img in enumerate(image_list):
    img_path = test_image_list[i]
    image_path = test_image_list[i].split('\\')[1]
    # 读取图片
    raw_img = image.load_img(img_path, target_size=(height, width))
    X_test = np.zeros((1, height, width, 3), dtype=np.float32)
    X_test[0] = image.img_to_array(raw_img)
    # 预测
    predict = model.predict(X_test)
    n = 62  # 大列表中几个数据组成一个小列表
    arr = []
    arr.append(predict[0][0:62])
    arr.append(predict[0][62:124])
    arr.append(predict[0][124:186])
    arr.append(predict[0][186:248])
    predictions = []
    predictions.append(np.argmax(arr[0]))
    predictions.append(np.argmax(arr[1]))
    predictions.append(np.argmax(arr[2]))
    predictions.append(np.argmax(arr[3]))
    # predictions = np.argmax(predict, axis=1)
    # 标签字典
    keys = range(62)
    label_dict = dict(zip(keys, captcha_word))
    result = ''.join([label_dict[pred] for pred in predictions])
    image_count = image_count + 1
    logger.debug("预测的结果为" + result)
    P = pd.DataFrame([[image_path,result]],columns=['ID','label'])
    test_data = test_data.append(P)

2019-11-06 16:31:33,506 : DEBUG : 预测的结果为s1O2
2019-11-06 16:31:33,575 : DEBUG : 预测的结果为8bud
2019-11-06 16:31:33,603 : DEBUG : 预测的结果为g16l
2019-11-06 16:31:33,629 : DEBUG : 预测的结果为hFSU
2019-11-06 16:31:33,641 : DEBUG : 预测的结果为whwJ
2019-11-06 16:31:33,662 : DEBUG : 预测的结果为n6jF
2019-11-06 16:31:33,675 : DEBUG : 预测的结果为oryN
2019-11-06 16:31:33,685 : DEBUG : 预测的结果为IpgW
2019-11-06 16:31:33,696 : DEBUG : 预测的结果为fT4M
2019-11-06 16:31:33,708 : DEBUG : 预测的结果为20du
2019-11-06 16:31:33,719 : DEBUG : 预测的结果为qAMs
2019-11-06 16:31:33,730 : DEBUG : 预测的结果为QGNM
2019-11-06 16:31:33,741 : DEBUG : 预测的结果为FmOG
2019-11-06 16:31:33,752 : DEBUG : 预测的结果为Q9k3
2019-11-06 16:31:33,763 : DEBUG : 预测的结果为wkWe
2019-11-06 16:31:33,774 : DEBUG : 预测的结果为JA0z
2019-11-06 16:31:33,785 : DEBUG : 预测的结果为NP32
2019-11-06 16:31:33,795 : DEBUG : 预测的结果为mp21
2019-11-06 16:31:33,807 : DEBUG : 预测的结果为ZhFK
2019-11-06 16:31:33,833 : DEBUG : 预测的结果为WAkN
2019-11-06 16:31:33,849 : DEBUG : 预测的结果为0lDA
2019-11-06 16:31:33,863 : DEBUG : 预测的结果为DM4r
2019-11-06

2019-11-06 16:31:35,884 : DEBUG : 预测的结果为80Yz
2019-11-06 16:31:35,902 : DEBUG : 预测的结果为0eBQ
2019-11-06 16:31:35,915 : DEBUG : 预测的结果为rr0M
2019-11-06 16:31:35,926 : DEBUG : 预测的结果为vwZ2
2019-11-06 16:31:35,937 : DEBUG : 预测的结果为Xsu0
2019-11-06 16:31:35,948 : DEBUG : 预测的结果为SEUs
2019-11-06 16:31:35,959 : DEBUG : 预测的结果为HV0X
2019-11-06 16:31:35,971 : DEBUG : 预测的结果为tnXd
2019-11-06 16:31:35,981 : DEBUG : 预测的结果为qsYT
2019-11-06 16:31:35,993 : DEBUG : 预测的结果为lM7I
2019-11-06 16:31:36,004 : DEBUG : 预测的结果为ohi6
2019-11-06 16:31:36,014 : DEBUG : 预测的结果为ATpI
2019-11-06 16:31:36,026 : DEBUG : 预测的结果为3dmQ
2019-11-06 16:31:36,037 : DEBUG : 预测的结果为4knz
2019-11-06 16:31:36,048 : DEBUG : 预测的结果为1FyZ
2019-11-06 16:31:36,060 : DEBUG : 预测的结果为E0kC
2019-11-06 16:31:36,070 : DEBUG : 预测的结果为VOJ2
2019-11-06 16:31:36,081 : DEBUG : 预测的结果为uTtV
2019-11-06 16:31:36,092 : DEBUG : 预测的结果为NLBp
2019-11-06 16:31:36,102 : DEBUG : 预测的结果为oegO
2019-11-06 16:31:36,128 : DEBUG : 预测的结果为Ueft
2019-11-06 16:31:36,139 : DEBUG : 预测的结果为yjIa
2019-11-06

2019-11-06 16:31:38,198 : DEBUG : 预测的结果为BmIZ
2019-11-06 16:31:38,211 : DEBUG : 预测的结果为e4Gq
2019-11-06 16:31:38,229 : DEBUG : 预测的结果为dDJs
2019-11-06 16:31:38,242 : DEBUG : 预测的结果为rIm6
2019-11-06 16:31:38,253 : DEBUG : 预测的结果为Hwtc
2019-11-06 16:31:38,264 : DEBUG : 预测的结果为N9k8
2019-11-06 16:31:38,275 : DEBUG : 预测的结果为4VBx
2019-11-06 16:31:38,286 : DEBUG : 预测的结果为dyT3
2019-11-06 16:31:38,296 : DEBUG : 预测的结果为P6N7
2019-11-06 16:31:38,314 : DEBUG : 预测的结果为YaZQ
2019-11-06 16:31:38,326 : DEBUG : 预测的结果为uS3m
2019-11-06 16:31:38,336 : DEBUG : 预测的结果为ofJ0
2019-11-06 16:31:38,347 : DEBUG : 预测的结果为qYLx
2019-11-06 16:31:38,363 : DEBUG : 预测的结果为rUTq
2019-11-06 16:31:38,377 : DEBUG : 预测的结果为W7i2
2019-11-06 16:31:38,387 : DEBUG : 预测的结果为WFpO
2019-11-06 16:31:38,398 : DEBUG : 预测的结果为Gzqr
2019-11-06 16:31:38,411 : DEBUG : 预测的结果为UNwR
2019-11-06 16:31:38,421 : DEBUG : 预测的结果为qamq
2019-11-06 16:31:38,432 : DEBUG : 预测的结果为iul6
2019-11-06 16:31:38,444 : DEBUG : 预测的结果为LhjM
2019-11-06 16:31:38,454 : DEBUG : 预测的结果为vgL5
2019-11-06

2019-11-06 16:31:40,653 : DEBUG : 预测的结果为2NnB
2019-11-06 16:31:40,663 : DEBUG : 预测的结果为AQ9l
2019-11-06 16:31:40,676 : DEBUG : 预测的结果为Xcda
2019-11-06 16:31:40,693 : DEBUG : 预测的结果为Qp4X
2019-11-06 16:31:40,707 : DEBUG : 预测的结果为CA2U
2019-11-06 16:31:40,716 : DEBUG : 预测的结果为qjYe
2019-11-06 16:31:40,733 : DEBUG : 预测的结果为35pa
2019-11-06 16:31:40,746 : DEBUG : 预测的结果为DEnE
2019-11-06 16:31:40,758 : DEBUG : 预测的结果为mx6V
2019-11-06 16:31:40,769 : DEBUG : 预测的结果为zcfW
2019-11-06 16:31:40,779 : DEBUG : 预测的结果为a3LM
2019-11-06 16:31:40,790 : DEBUG : 预测的结果为ecPv
2019-11-06 16:31:40,800 : DEBUG : 预测的结果为bvOC
2019-11-06 16:31:40,812 : DEBUG : 预测的结果为6bqu
2019-11-06 16:31:40,827 : DEBUG : 预测的结果为Pwt0
2019-11-06 16:31:40,840 : DEBUG : 预测的结果为NsMD
2019-11-06 16:31:40,849 : DEBUG : 预测的结果为SD5Y
2019-11-06 16:31:40,865 : DEBUG : 预测的结果为kIiA
2019-11-06 16:31:40,878 : DEBUG : 预测的结果为RL25
2019-11-06 16:31:40,889 : DEBUG : 预测的结果为SpAo
2019-11-06 16:31:40,899 : DEBUG : 预测的结果为XYeb
2019-11-06 16:31:40,911 : DEBUG : 预测的结果为mUk6
2019-11-06

2019-11-06 16:31:43,052 : DEBUG : 预测的结果为NcsD
2019-11-06 16:31:43,062 : DEBUG : 预测的结果为prDe
2019-11-06 16:31:43,074 : DEBUG : 预测的结果为2uGe
2019-11-06 16:31:43,085 : DEBUG : 预测的结果为fCnR
2019-11-06 16:31:43,096 : DEBUG : 预测的结果为OPC2
2019-11-06 16:31:43,108 : DEBUG : 预测的结果为z9f2
2019-11-06 16:31:43,119 : DEBUG : 预测的结果为Var7
2019-11-06 16:31:43,130 : DEBUG : 预测的结果为CDAW
2019-11-06 16:31:43,142 : DEBUG : 预测的结果为AhcL
2019-11-06 16:31:43,152 : DEBUG : 预测的结果为lxkJ
2019-11-06 16:31:43,163 : DEBUG : 预测的结果为CaAJ
2019-11-06 16:31:43,175 : DEBUG : 预测的结果为NB2Z
2019-11-06 16:31:43,186 : DEBUG : 预测的结果为KKTW
2019-11-06 16:31:43,196 : DEBUG : 预测的结果为88f2
2019-11-06 16:31:43,208 : DEBUG : 预测的结果为V4yr
2019-11-06 16:31:43,223 : DEBUG : 预测的结果为6HVr
2019-11-06 16:31:43,235 : DEBUG : 预测的结果为BJqp
2019-11-06 16:31:43,246 : DEBUG : 预测的结果为qrFk
2019-11-06 16:31:43,258 : DEBUG : 预测的结果为NzK2
2019-11-06 16:31:43,269 : DEBUG : 预测的结果为0VxV
2019-11-06 16:31:43,279 : DEBUG : 预测的结果为IKJ5
2019-11-06 16:31:43,291 : DEBUG : 预测的结果为grno
2019-11-06

...

2019-11-06 16:32:24,791 : DEBUG : 预测的结果为cn8h
2019-11-06 16:32:24,801 : DEBUG : 预测的结果为ub4m
2019-11-06 16:32:24,812 : DEBUG : 预测的结果为dEgh
2019-11-06 16:32:24,823 : DEBUG : 预测的结果为dRFL
2019-11-06 16:32:24,834 : DEBUG : 预测的结果为Bkmo
2019-11-06 16:32:24,844 : DEBUG : 预测的结果为UrKe
2019-11-06 16:32:24,854 : DEBUG : 预测的结果为xDwM
2019-11-06 16:32:24,865 : DEBUG : 预测的结果为o3ny
2019-11-06 16:32:24,875 : DEBUG : 预测的结果为r7V3
2019-11-06 16:32:24,885 : DEBUG : 预测的结果为Ar9J
2019-11-06 16:32:24,896 : DEBUG : 预测的结果为J3aA
2019-11-06 16:32:24,905 : DEBUG : 预测的结果为Vgy3
2019-11-06 16:32:24,916 : DEBUG : 预测的结果为BGqV
2019-11-06 16:32:24,927 : DEBUG : 预测的结果为QRTy
2019-11-06 16:32:24,937 : DEBUG : 预测的结果为PeYL
2019-11-06 16:32:24,948 : DEBUG : 预测的结果为Kw88
2019-11-06 16:32:24,958 : DEBUG : 预测的结果为V0VQ
2019-11-06 16:32:24,969 : DEBUG : 预测的结果为UnMz
2019-11-06 16:32:24,979 : DEBUG : 预测的结果为kVq2
2019-11-06 16:32:24,989 : DEBUG : 预测的结果为sIkV
2019-11-06 16:32:25,000 : DEBUG : 预测的结果为x2a2
2019-11-06 16:32:25,010 : DEBUG : 预测的结果为Fyl4
2019-11-06

2019-11-06 16:32:26,963 : DEBUG : 预测的结果为KLzC
2019-11-06 16:32:26,974 : DEBUG : 预测的结果为Pw8s
2019-11-06 16:32:26,983 : DEBUG : 预测的结果为crRl
2019-11-06 16:32:26,994 : DEBUG : 预测的结果为alN0
2019-11-06 16:32:27,003 : DEBUG : 预测的结果为Iocc
2019-11-06 16:32:27,014 : DEBUG : 预测的结果为LxUm
2019-11-06 16:32:27,023 : DEBUG : 预测的结果为lofg
2019-11-06 16:32:27,034 : DEBUG : 预测的结果为QHY8
2019-11-06 16:32:27,044 : DEBUG : 预测的结果为Sp21
2019-11-06 16:32:27,054 : DEBUG : 预测的结果为RNic
2019-11-06 16:32:27,064 : DEBUG : 预测的结果为SKk4
2019-11-06 16:32:27,073 : DEBUG : 预测的结果为qHf9
2019-11-06 16:32:27,083 : DEBUG : 预测的结果为64z3
2019-11-06 16:32:27,100 : DEBUG : 预测的结果为GgiR
2019-11-06 16:32:27,113 : DEBUG : 预测的结果为5TQz
2019-11-06 16:32:27,122 : DEBUG : 预测的结果为wOWN
2019-11-06 16:32:27,133 : DEBUG : 预测的结果为B93f
2019-11-06 16:32:27,143 : DEBUG : 预测的结果为hEzq
2019-11-06 16:32:27,153 : DEBUG : 预测的结果为R21n
2019-11-06 16:32:27,164 : DEBUG : 预测的结果为R9MX
2019-11-06 16:32:27,174 : DEBUG : 预测的结果为1itT
2019-11-06 16:32:27,185 : DEBUG : 预测的结果为XgDx
2019-11-06

2019-11-06 16:32:29,016 : DEBUG : 预测的结果为SNfp
2019-11-06 16:32:29,026 : DEBUG : 预测的结果为HFe2
2019-11-06 16:32:29,037 : DEBUG : 预测的结果为i8l3
2019-11-06 16:32:29,059 : DEBUG : 预测的结果为oF1X
2019-11-06 16:32:29,075 : DEBUG : 预测的结果为BkEA
2019-11-06 16:32:29,087 : DEBUG : 预测的结果为NeUf
2019-11-06 16:32:29,097 : DEBUG : 预测的结果为Ig8B
2019-11-06 16:32:29,108 : DEBUG : 预测的结果为741K
2019-11-06 16:32:29,125 : DEBUG : 预测的结果为rFxu
2019-11-06 16:32:29,138 : DEBUG : 预测的结果为fCZu
2019-11-06 16:32:29,148 : DEBUG : 预测的结果为qbuR
2019-11-06 16:32:29,159 : DEBUG : 预测的结果为oXuU
2019-11-06 16:32:29,170 : DEBUG : 预测的结果为ui4X
2019-11-06 16:32:29,180 : DEBUG : 预测的结果为tZZW
2019-11-06 16:32:29,191 : DEBUG : 预测的结果为2KQY
2019-11-06 16:32:29,201 : DEBUG : 预测的结果为3b4Q
2019-11-06 16:32:29,212 : DEBUG : 预测的结果为0z2T
2019-11-06 16:32:29,222 : DEBUG : 预测的结果为6Ldh
2019-11-06 16:32:29,233 : DEBUG : 预测的结果为AxXL
2019-11-06 16:32:29,243 : DEBUG : 预测的结果为O3AQ
2019-11-06 16:32:29,253 : DEBUG : 预测的结果为GyGY
2019-11-06 16:32:29,264 : DEBUG : 预测的结果为YGUg
2019-11-06

2019-11-06 16:32:31,054 : DEBUG : 预测的结果为zNAs
2019-11-06 16:32:31,066 : DEBUG : 预测的结果为QfcW
2019-11-06 16:32:31,076 : DEBUG : 预测的结果为QQk8
2019-11-06 16:32:31,086 : DEBUG : 预测的结果为b8TD
2019-11-06 16:32:31,096 : DEBUG : 预测的结果为tUaV
2019-11-06 16:32:31,111 : DEBUG : 预测的结果为Dk0d
2019-11-06 16:32:31,124 : DEBUG : 预测的结果为unma
2019-11-06 16:32:31,134 : DEBUG : 预测的结果为KdVe
2019-11-06 16:32:31,144 : DEBUG : 预测的结果为ZRzo
2019-11-06 16:32:31,155 : DEBUG : 预测的结果为PXXa
2019-11-06 16:32:31,166 : DEBUG : 预测的结果为mge2
2019-11-06 16:32:31,176 : DEBUG : 预测的结果为VHWs
2019-11-06 16:32:31,186 : DEBUG : 预测的结果为lsGz
2019-11-06 16:32:31,197 : DEBUG : 预测的结果为cZSe
2019-11-06 16:32:31,207 : DEBUG : 预测的结果为b07e
2019-11-06 16:32:31,217 : DEBUG : 预测的结果为nQN4
2019-11-06 16:32:31,228 : DEBUG : 预测的结果为J8hj
2019-11-06 16:32:31,238 : DEBUG : 预测的结果为6CGS
2019-11-06 16:32:31,248 : DEBUG : 预测的结果为n0IY
2019-11-06 16:32:31,257 : DEBUG : 预测的结果为Ut6K
2019-11-06 16:32:31,267 : DEBUG : 预测的结果为ekY4
2019-11-06 16:32:31,277 : DEBUG : 预测的结果为4NzA
2019-11-06

2019-11-06 16:32:33,110 : DEBUG : 预测的结果为fKUq
2019-11-06 16:32:33,120 : DEBUG : 预测的结果为GMJG
2019-11-06 16:32:33,131 : DEBUG : 预测的结果为orwc
2019-11-06 16:32:33,141 : DEBUG : 预测的结果为xFn2
2019-11-06 16:32:33,152 : DEBUG : 预测的结果为t0GX
2019-11-06 16:32:33,163 : DEBUG : 预测的结果为SFYC
2019-11-06 16:32:33,174 : DEBUG : 预测的结果为sdZv
2019-11-06 16:32:33,184 : DEBUG : 预测的结果为vHCz
2019-11-06 16:32:33,194 : DEBUG : 预测的结果为xcjL
2019-11-06 16:32:33,204 : DEBUG : 预测的结果为MMVN
2019-11-06 16:32:33,214 : DEBUG : 预测的结果为FVlc
2019-11-06 16:32:33,224 : DEBUG : 预测的结果为Re08
2019-11-06 16:32:33,235 : DEBUG : 预测的结果为morE
2019-11-06 16:32:33,244 : DEBUG : 预测的结果为0gVr
2019-11-06 16:32:33,255 : DEBUG : 预测的结果为B0zB
2019-11-06 16:32:33,265 : DEBUG : 预测的结果为ksxs
2019-11-06 16:32:33,275 : DEBUG : 预测的结果为5MVg
2019-11-06 16:32:33,286 : DEBUG : 预测的结果为d9t7
2019-11-06 16:32:33,296 : DEBUG : 预测的结果为wKTD
2019-11-06 16:32:33,306 : DEBUG : 预测的结果为x2rt
2019-11-06 16:32:33,324 : DEBUG : 预测的结果为kxS7
2019-11-06 16:32:33,336 : DEBUG : 预测的结果为PnGw
2019-11-06

In [49]:
test_data.to_csv('20191106_cnn3.csv',index=None)

In [ ]:
import cv2
import numpy as np

line_num = 5
#图片个数
pic_num = 2
path = "validate/"


def randcolor():
    return (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
    # return (0, 0, 0)


def randchar():
    return np.random.choice([chr(np.random.randint(65, 91)), chr(np.random.randint(97, 123)) ,np.random.randint(0, 9)])


def randpos(x_start, x_end, y_start, y_end):
    return (np.random.randint(x_start, x_end),
            np.random.randint(y_start, y_end))


img_heigth = 60
img_width = 240
for i in range(pic_num):
    img_name = ""
    # 生成一个随机矩阵，randint(low[, high, size, dtype])
    img = np.random.randint(np.random.randint(50, 100), np.random.randint(100, 150), (img_heigth, img_width, 3), np.uint8)
    # 显示图像
    cv2.imshow("ranImg",img)
    x_pos = 0
    y_pos = 25
    for i in range(4):
        char = randchar()
        while char == 'i' or char == 'I' or char == 'l' or char == 'L' or char == 'o' or char == 'O':
            char = randchar()
        img_name += char
        #各参数依次是：图片，添加的文字，左上角坐标，字体，字体大小，颜色，字体粗细
        cv2.putText(img, char,
                    (np.random.randint(x_pos, x_pos + 20), np.random.randint(y_pos, y_pos + 20)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.2,
                    randcolor(),
                    3,
                    cv2.LINE_AA)
        x_pos += 45

    cv2.imshow("res",img)

    # 添加线段
    for i in range(line_num):
        img = cv2.line(img,
                       randpos(0, img_width, 0, img_heigth),
                       randpos(0, img_width, 0, img_heigth),
                       randcolor(),
                       np.random.randint(1, 2))

    cv2.imshow("line",img)
    cv2.imwrite(path + img_name + ".jpg", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()